# Monitoring Environmental Waste Utilization Scores
# A guide for generating EWU-Dashboards
**Sarah Schmidt & David Laner**  
*January 2023*

## 1. Configuration

### 1.1 Import packages

In [5]:
import os
import pandas as pd
import numpy as np
import brightway2 as bw
import premise
from mycolorpy import colorlist as mcp
import string

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [6]:
# create a list with MS Excel column indices (A, B, ..., Z, AA, AB, ...)
# will be used for preparing calculations in MS Excel
alphabet = list(string.ascii_uppercase)
excel_cols=[]

for i in range(100):
    n=0
    j=i
    while j-len(alphabet)>=0:
        j=j-len(alphabet)
        n=n+1
    if n>0:
        col=alphabet[n-1]+alphabet[j]
    else:
        col=alphabet[j]
    excel_cols.append(col)

In [7]:
conversion_factors={'kg':1,
                    't':1e3,
                    'kt':1e6,
                    'g':1e-3,
                    'Mg':1e3,
                    'Gg':1e6}

### 1.2 Gather case study-specific information

In [1762]:
# read various case study specific information from input data file (ExcelTool_GeneratorInput_Template.xlsx)
general_info=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='GeneralInformation', index_col=0)
general_info=general_info[general_info.columns[0]].to_dict()

Pathway_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
Pathway_codes=Pathway_codes[Pathway_codes['Pathway tag'].notnull()]['Activity code'].to_list()
Pathway_names=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
Pathway_names=Pathway_names[Pathway_names['Pathway tag'].notnull()]['Pathway tag'].to_list()

WasteGen_code=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
WasteGen_code=WasteGen_code[WasteGen_code['Waste generation'].notnull()]['Activity code'].iloc[0]

act_codes=[WasteGen_code,*Pathway_codes]
act_names=[general_info['Acronym'],*Pathway_names]

# foreground system
activity_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
foreground_system_codes=activity_df[activity_df['Foreground'].notnull()]['Activity code'].to_list()

# number of foreground system scenarios
n_scenarios=general_info['Maximum number of foreground system scenarios']

# materials for calculation of the environmental impact of materials
material_names=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='EnvironmentalValue')
material_names=material_names['Material Name'].to_list()
material_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='EnvironmentalValue')
material_codes=material_codes['Material Code'].to_list()
activity_material_dict={}
for i in activity_df.index:
    if pd.isnull(activity_df.loc[i,'Material tag'])==False:
        activity_material_dict[activity_df.loc[i,'Activity code']]=activity_df.loc[i,'Material tag']
        
waste_utilization_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
waste_utilization_codes=waste_utilization_codes[pd.isnull(waste_utilization_codes['Waste utilization'])==False]['Activity code'].to_list()

# activity tags for contribution analysis
activity_tags=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
activity_tags=activity_tags['Activity tag'].to_list()
activity_tags=[tag for tag in activity_tags if pd.isnull(tag)==False]
activity_tags=list(set(activity_tags))
activity_tags.append('Others')

# waste quantity in kg
WQ_kg=general_info['Waste quantity']*conversion_factors[general_info['Unit']]

### 1.3 Setup and import of databases

#### 1.3.1 Setup

In [9]:
# create a new project or open an existing project
bw.projects.set_current(general_info['Project name'])

In [10]:
# creates the database "biosphere 3"
bw.bw2setup() 
biosphere = bw.Database("biosphere3")

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/16/2023 15:59:58
  Finished: 01/16/2023 15:59:58
  Total time elapsed: 00:00:00
  CPU %: 6.90
  Memory %: 1.66
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: fix_ecoinvent_38_lcia_implementation
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 5 strategies in 1.77 seconds
Wrote 975 LCIA methods with 254388 characterization factors
Creating core data migrations



#### 1.3.2 Ecoinvent

In [11]:
# import of the ecoinvent database
db_default_name=general_info['Database name']+'_default'
if db_default_name in bw.databases:
    print("Database has already been imported.")
    eidb_default = bw.Database(db_default_name)
else:
    # mind that the ecoinvent file must be unzipped; then: path to the datasets subfolder
    fpeidbcut = r"{}".format(general_info['Database file path'])
    # the "r" makes sure that the path is read as a string - especially useful when you have spaces in your string
    eidbcut = bw.SingleOutputEcospold2Importer(fpeidbcut, general_info['Database name']+'_default')
    eidbcut
    eidbcut.apply_strategies()
    eidbcut.statistics()
    eidb_default=eidbcut.write_database()

Extracting XML data from 19128 datasets
Extracted 19128 datasets in 79.63 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: 

Writing activities to SQLite3 database:


19128 datasets
621719 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:05


Title: Writing activities to SQLite3 database:
  Started: 01/16/2023 16:03:19
  Finished: 01/16/2023 16:04:24
  Total time elapsed: 00:01:05
  CPU %: 23.30
  Memory %: 21.99
Created database: ecoinvent 3.7.1_cutoff_ecoSpold02_default


In [12]:
# copy of the unmodified version of the ecoinvent database
if general_info['Database name'] in bw.databases:
    print("Database has already been imported.")
else:
    eidb_default.copy(general_info['Database name'])
eidb = bw.Database(general_info['Database name'])

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:04


Title: Writing activities to SQLite3 database:
  Started: 01/16/2023 16:08:20
  Finished: 01/16/2023 16:09:24
  Total time elapsed: 00:01:04
  CPU %: 20.00
  Memory %: 55.29


#### 1.3.3 Prospective Databases (premise)

In [13]:
# gather information which prospective scenarios shall be created
premise_scenarios=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='ProspectiveScenarios', 
                                skiprows=3, nrows=8)
premise_update=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='ProspectiveScenarios', 
                                index_col=0, skiprows=15, nrows=9, usecols='A:B')
premise_update=premise_update[premise_update['Update']=='yes']

In [14]:
# read encryption key 
# (to be requested from the premise library maintainers if you want ot use default scenarios included in `premise`)
encryption_key=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='ProspectiveScenarios', 
                         usecols = "B", header = 0, nrows=0).columns[0]

In [26]:
if general_info['Generate background system scenarios']=='yes':
    if (general_info ['Type of background system scenarios']=='premise') or (general_info ['Type of background system scenarios']=='manual + premise'):
        
        premise_scenarios_dictlist=[]
        premise_scenario_names_list=[]

        for i in premise_scenarios.index:
            for c in premise_scenarios.columns[3:]:
                if premise_scenarios.loc[i,c]=='x':
                    scenario_dict={"model":premise_scenarios.loc[i,'IAM'], 
                                   "pathway":premise_scenarios.loc[i,'SSP']+'-'+premise_scenarios.loc[i,'RCP'], 
                                   "year":c}
                    scenario_name=premise_scenarios.loc[i,'IAM']+str(c)+'_'+premise_scenarios.loc[i,'SSP']+'-'+premise_scenarios.loc[i,'RCP']
                    premise_scenarios_dictlist.append(scenario_dict)
                    premise_scenario_names_list.append(scenario_name)
       
                    if scenario_name not in bw.databases:
                        ndb = premise.NewDatabase(
                            scenarios=[scenario_dict],
                            source_db=eidb_default.name, # name of the database in the BW2 project. Must be a string.
                            source_version=general_info['Database version'], # version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
                            key=encryption_key,# <-- decryption key
                            quiet=True
                            # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
                            )


                        if len(premise_update)==8:
                            ndb.update_all()
                        else:
                            if 'Electricity' in premise_update.index:
                                ndb.update_electricity()
                            if 'Cement' in premise_update.index:
                                ndb.update_cement()
                            if 'Steel' in premise_update.index:
                                ndb.update_steel()
                            if 'Fuels' in premise_update.index:
                                ndb.update_fuels()
                            if 'Cars' in premise_update.index:
                                ndb.update_cars()
                            if 'Trucks' in premise_update.index:
                                ndb.update_trucks()
                            if 'Two wheelers' in premise_update.index:
                                ndb.update_two_wheelers()
                            if 'Buses' in premise_update.index:
                                ndb.update_buses()

                        ndb.write_db_to_brightway(name=[scenario_name])    

premise v.(1, 3, 0)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

100%|██████████████████████████████████████████████████████████████| 19128/19128 [00:00<00:00, 87866.15it/s]


Adding exchange data to activities


100%|████████████████████████████████████████████████████████████| 621719/621719 [00:51<00:00, 12112.23it/s]


Filling out exchange data


...

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:32


Title: Writing activities to SQLite3 database:
  Started: 01/16/2023 18:50:35
  Finished: 01/16/2023 18:53:08
  Total time elapsed: 00:02:32
  CPU %: 22.60
  Memory %: 48.36
Created database: image2030_SSP2-RCP19
premise v.(1, 3, 0)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from p

Done!

////////////////////////////// FUELS ///////////////////////////////
Generate region-specific direct air capture processes.
Generate region-specific hydrogen production pathways.
Generate region-specific hydrogen supply chains.
Generate region-specific biogas and syngas supply chains.
Generate region-specific synthetic fuel supply chains.
Generate region-specific biofuel supply chains.
Generate new fuel markets.
--> petrol, unleaded
--> petrol, low-sulfur
--> diesel, low-sulfur
--> diesel
Log of deleted fuel markets saved in C:\Users\SarahSchmidt\anaconda3\envs\MoEWe2\Lib\site-packages\premise\data\logs
Log of created fuel markets saved in C:\Users\SarahSchmidt\anaconda3\envs\MoEWe2\Lib\site-packages\premise\data\logs
Done!
Write new database(s) to Brightway2.
Prepare database 1.
- check for duplicates...
One or multiple duplicates detected. Removing them...
- check for values format...
- relinking exchanges...
Done!
46233 datasets
2030722 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:49


Title: Writing activities to SQLite3 database:
  Started: 01/16/2023 20:11:07
  Finished: 01/16/2023 20:13:57
  Total time elapsed: 00:02:49
  CPU %: 23.40
  Memory %: 51.63
Created database: image2035_SSP2-RCP19


#### 1.3.5 Overview of databases to be included in the EWU-Dashboard

In [40]:
dbs=[eidb]

db_names_dict={}
db_names_dict['default']=eidb.name
n=1

if general_info['Generate background system scenarios']=='yes':
    if (general_info ['Type of background system scenarios']=='premise') or (general_info ['Type of background system scenarios']=='manual + premise'):
        for scenario in premise_scenario_names_list:
            dbs.append(bw.Database(scenario))
            db='db'+str(n)
            db_names_dict[db]=scenario
            n=n+1

db_names=[db for db in db_names_dict.keys()]

### 1.4 LCIA Methods, Normalization Factors & Weighting Factors

**Methods**

In [29]:
LCIAmethod_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='LCIA_Methods')
LCIA_method_names=LCIAmethod_df['Acronym'].to_list()

In [30]:
LCIA_methods=[]
for i in LCIAmethod_df.index:
    method=[m for m in bw.methods if m[0]==LCIAmethod_df.loc[i, 'Method_Part1'] and
                                     m[1]==LCIAmethod_df.loc[i, 'Method_Part2'] and
                                     m[2]==LCIAmethod_df.loc[i, 'Method_Part3']][0]
    LCIA_methods.append(method)

In [31]:
LCIAmethod_sheet_name='LCIA_Methods'

In [32]:
LCIAmethod_df=LCIAmethod_df.set_index('Acronym', drop=True)

**Normalization Factors**

In [33]:
nfs=LCIAmethod_df['Normalization Factor'].to_list()

**Weighting Factors**

In [34]:
Weighting_sheet_name='Weighting'

In [35]:
Weighting=pd.read_excel("ExcelTool_GeneratorInput_Template.xlsx", sheet_name="Weighting", index_col=0)

## 2. Calculations

### 2.1 Life Cycle Inventory

#### 2.1.1 Activities

In [1690]:
activity_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')

In [41]:
for db in dbs:
    print(db.name)
    for i in activity_df.index:
        if len([act for act in db if act['code']==activity_df.loc[i,'Activity code']])==0:
            activity = db.new_activity(code = activity_df.loc[i,'Activity code'], name = activity_df.loc[i,'Activity name'], unit = activity_df.loc[i,'Unit'], location = activity_df.loc[i,'Location'])
            activity.save()
        else:
            activity=[act for act in db if act['code']==activity_df.loc[i,'Activity code']][0]
            if activity['name']!=activity_df.loc[i,'Activity name']:
                print("Error: Activity name", activity['name'], activity_df.loc[i,'Activity name'])
            if activity['location']!=activity_df.loc[i,'Location']:
                print("Error: Activity location", activity['location'], activity_df.loc[i,'Location'])
            if activity['unit']!=activity_df.loc[i,'Unit']:
                print("Error: Activity unit", activity['unit'], activity_df.loc[i,'Unit'])

ecoinvent 3.7.1_cutoff_ecoSpold02
image2025_SSP2-RCP19
image2030_SSP2-RCP19
image2035_SSP2-RCP19


#### 2.2.2 Exchanges

In [410]:
exchanges_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Exchanges')

In [43]:
for db in dbs:
    print(db.name)
    for actcode in exchanges_df['activity code'].unique():
        act_exchanges_df=exchanges_df[exchanges_df['activity code']==actcode]
        act=[act for act in db if act['code']==actcode][0]
        act.exchanges().delete()
        for i in act_exchanges_df.index:
            if (act_exchanges_df.loc[i,'type']=='technosphere') or (act_exchanges_df.loc[i,'type']=='production'):
                exc_input=[act for act in db if act['code']==act_exchanges_df.loc[i,'input code']][0]
            if act_exchanges_df.loc[i,'type']=='biosphere':
                exc_input=[act for act in biosphere if act['code']==act_exchanges_df.loc[i,'input code']][0]
            act.new_exchange(input = exc_input.key, amount = act_exchanges_df.loc[i,'amount'], 
                                 unit = act_exchanges_df.loc[i,'input unit'], type = act_exchanges_df.loc[i,'type']).save() 
            act.save()             

            exc=[exc for exc in act.exchanges() if exc['input']==exc_input.key][0]

            #import material flow tag
            if (exc_input['code'] in activity_df['Activity code']) & (exc_input['unit'] == 'kilogram'):
                exc['tag']='material flow'
                exc.save()
                act.save()

ecoinvent 3.7.1_cutoff_ecoSpold02
image2025_SSP2-RCP19
image2030_SSP2-RCP19
image2035_SSP2-RCP19


#### 1.3.4 Manual Background Scenarios

In [44]:
background_scenarios=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', 
                                   sheet_name='BackgroundScenarios')

,background scenario,database,input,input code,input unit,input location / biosphere category,activity,activity code,activity location,amount,type
0,heat+cement,image2025_SSP2-RCP19,"avoided emissions, CO2",avoidedCO2,kilogram,GLO,"substitution of fuels in cement kilns, mixed p...",CemKiln-FuelSubst-MP,DE,-0.123707,technosphere
1,heat+cement,image2025_SSP2-RCP19,"Carbon dioxide, fossil",aa7cac3a-3625-41d4-bc54-33e2cf11ec46,kilogram,"('air', 'non-urban air or from high stacks')","substitution of fuels in cement kilns, mixed p...",CemKiln-FuelSubst-MP,DE,2.122120,biosphere
2,heat+cement,image2025_SSP2-RCP19,market for wood pellet,9c663a9f4aa6d200ced09fd7a3ed7f98,kilogram,RER,"substitution of fuels in cement kilns, mixed p...",CemKiln-FuelSubst-MP,DE,-1.998235,technosphere
3,heat+cement,image2025_SSP2-RCP19,market for hard coal,a72db9f7595317eda5100fff717c584d,kilogram,RoW,"substitution of fuels in cement kilns, mixed p...",CemKiln-FuelSubst-MP,DE,0.000000,technosphere
4,heat+cement,image2025_SSP2-RCP19,"avoided emissions, CO2",avoidedCO2,kilogram,GLO,"substitution of fuels in cement kilns, polyeth...",CemKiln-FuelSubst-PE,DE,-0.162417,technosphere
...,...,...,...,...,...,...,...,...,...,...,...
91,heat+cement,image2035_SSP2-RCP19,"heat and power co-generation, natural gas, com...",ba05d6265dc78e070c8ce7ae01828dfe,megajoule,RoW,"market for heat, mix, export countries",heat_mix_export,GLO,0.000000,technosphere
92,heat+cement,image2035_SSP2-RCP19,"natural gas, burned in gas motor, for storage",5782508bdc29a389625856679cb35c82,megajoule,RoW,"market for heat, mix, export countries",heat_mix_export,GLO,0.000000,technosphere
93,heat+cement,image2035_SSP2-RCP19,"heat production, at hot water tank, solar+elec...",374031b1fbc7d00b316f69b007133d59,megajoule,CH,"market for heat, mix, export countries",heat_mix_export,GLO,0.027778,technosphere
94,heat+cement,image2035_SSP2-RCP19,"heat production, air-water heat pump 10kW",85577cff356913040aa8e0cb9dd39bd8,megajoule,RoW,"market for heat, mix, export countries",heat_mix_export,GLO,0.777778,technosphere


In [85]:
if general_info['Generate background system scenarios']=='yes':
    if (general_info ['Type of background system scenarios']=='manual') or (general_info ['Type of background system scenarios']=='manual + premise'):
        
        background_scenario_names_list=[]
        
        for bs in background_scenarios['background scenario'].unique():
            sub_df=background_scenarios[background_scenarios['background scenario']==bs]          
            for database in sub_df['database'].unique():
                db_name=database+'_'+bs
                if db_name not in bw.databases:
                    db=bw.Database(database)
                    db.copy(db_name)
                else:
                    print("Database has already been imported.")
                
                if db_name not in background_scenario_names_list:
                    background_scenario_names_list.append(db_name)
            
            
        if general_info['Handling of manual background system scenarios']=='adapt exchange amounts':
            for i in background_scenarios.index:
                db_name=background_scenarios.loc[i,'database']+'_'+background_scenarios.loc[i,'background scenario']
                bsdb=bw.Database(db_name)
                act=bsdb.get(background_scenarios.loc[i,'activity code'])
                exc=[exc for exc in act.exchanges() if exc.input.as_dict()['code']==background_scenarios.loc[i,'input code']][0]

                exc['amount']=background_scenarios.loc[i,'amount']
                exc.save()
                    
        if general_info['Handling of manual background system scenarios']=='new exchanges':
            for bs in background_scenarios['background scenario'].unique():
                sub_df=background_scenarios[background_scenarios['background scenario']==bs]
                for database in sub_df['database'].unique():
                    sub_sub_df=sub_df[sub_df['database']==database]
                    for actcode in sub_sub_df['activity code'].unique():
                        sub_sub_sub_df=sub_sub_df[sub_sub_df['activity code']==actcode]
                        db_name=database+'_'+bs
                        bsdb=bw.Database(db_name)
                        act=bsdb.get(actcode)
                        act.technosphere().delete()
                        act.biosphere().delete()
                        for i in sub_sub_sub_df.index:
                            if sub_sub_sub_df.loc[i,'type']=='technosphere':
                                input_act=bsdb.get(sub_sub_sub_df.loc[i,'input code'])
                            else:
                                input_act=biosphere.get(sub_sub_sub_df.loc[i,'input code'])
                            act.new_exchange(input=input_act.key,
                                            unit=input_act['unit'],
                                            amount=sub_sub_sub_df.loc[i,'amount'],
                                            type=sub_sub_sub_df.loc[i,'type']).save()
                            act.save()

Database has already been imported.
Database has already been imported.
Database has already been imported.


In [90]:
if general_info['Generate background system scenarios']=='yes':
    if (general_info ['Type of background system scenarios']=='manual') or (general_info['Type of background system scenarios']=='manual + premise'):
        for scenario in background_scenario_names_list:
            dbs.append(bw.Database(scenario))
            db='db'+str(n)
            db_names_dict[db]=scenario
            n=n+1
            
db_names=[db for db in db_names_dict.keys()]

### 2.2 Modular Life Cycle Assessment

In [1758]:
activity_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')

In [1763]:
foreground_system=[]
for actcode in foreground_system_codes:
    foreground_system.append(eidb.get(actcode))

In [1764]:
foreground_system_groups={}
for ac,actcode in enumerate(foreground_system_codes):
    foreground_system_groups[actcode]=ac

#### 2.2.1 LCI to DataFrame

In [1766]:
LCI_df=pd.DataFrame()
i=0

for act in foreground_system:   
    act_name=act.as_dict()['name']
    act_code=act.as_dict()['code']
    act_unit=act.as_dict()['unit']
    act_location=act.as_dict()['location']
    #production exchange
    LCI_df.loc[i,'input']=act_name
    LCI_df.loc[i,'input code']=act_code
    LCI_df.loc[i,'input unit']=act_unit
    LCI_df.loc[i,'input location']=act_location
    LCI_df.loc[i,'activity']=act_name
    LCI_df.loc[i,'activity code']=act_code
    LCI_df.loc[i,'activity location']=act_location
    if len([exc for exc in eidb.get(act_code).production()]) >0:
        LCI_df.loc[i,'amount']=[exc.amount for exc in eidb.get(act_code).production()][0]
    else:
        LCI_df.loc[i,'amount']=1
    LCI_df.loc[i,'type']='production'
    LCI_df.loc[i,'material flow']=False
    i=i+1

    #technosphere exchanges
    for exc in act.technosphere():
        LCI_df.loc[i,'input']=exc.input.as_dict()['name']
        LCI_df.loc[i,'input code']=exc.input.as_dict()['code']
        LCI_df.loc[i,'input unit']=exc.input.as_dict()['unit']
        LCI_df.loc[i,'input location']=exc.input.as_dict()['location']
        LCI_df.loc[i,'activity']=act_name
        LCI_df.loc[i,'activity code']=act_code
        LCI_df.loc[i,'activity location']=act_location
        LCI_df.loc[i,'amount']=exc.amount
        LCI_df.loc[i,'type']='technosphere'
        if exc.input.as_dict()['code'] in foreground_system_codes:
            LCI_df.loc[i,'material flow']=True
        else:
            LCI_df.loc[i,'material flow']=False
        i=i+1
        
    #biosphere exchanges
    for exc in act.biosphere():
        LCI_df.loc[i,'input']=exc.input.as_dict()['name']
        LCI_df.loc[i,'input code']=exc.input.as_dict()['code']
        LCI_df.loc[i,'input unit']=exc.input.as_dict()['unit']
        LCI_df.loc[i,'input location']=str(exc.input.as_dict()['categories'])
        LCI_df.loc[i,'activity']=act_name
        LCI_df.loc[i,'activity code']=act_code
        LCI_df.loc[i,'activity location']=act_location
        LCI_df.loc[i,'amount']=exc.amount
        LCI_df.loc[i,'type']='biosphere'
        LCI_df.loc[i,'material flow']=False
        i=i+1

LCI_df

,input,input code,input unit,input location,activity,activity code,activity location,amount,type,material flow
0,"thermochemical recycling, LDPE",ChemRecLDPE,kilogram,DE,"thermochemical recycling, LDPE",ChemRecLDPE,DE,-1.0000,production,False
1,"market for electricity, medium voltage",d20464840573665d431297c62b3cab90,kilowatt hour,DE,"thermochemical recycling, LDPE",ChemRecLDPE,DE,0.4430,technosphere,False
2,"market for hydrogen, liquid",a6b5e5e5a57b04a260b6529cf7c63f30,kilogram,RER,"thermochemical recycling, LDPE",ChemRecLDPE,DE,0.0140,technosphere,False
3,"market for lime, hydrated, packed",1db40f8b58995bec8ce3de08cf5cfded,kilogram,RER,"thermochemical recycling, LDPE",ChemRecLDPE,DE,0.0050,technosphere,False
4,market for naphtha,70e4783e7c9ca937a1d06048d7b91ef9,kilogram,RER,"thermochemical recycling, LDPE",ChemRecLDPE,DE,-0.5200,technosphere,False
...,...,...,...,...,...,...,...,...,...,...
3894,"treatment of municipal solid waste, incineration",73c9bb8a89bfa6026bce836e3efddb74,kilogram,RoW,"thermochemical recycling, MP",ChemRecMP_export,GLO,-0.0500,technosphere,False
3895,substitution of heat,heat-Substitution_export,megajoule,GLO,"thermochemical recycling, MP",ChemRecMP_export,GLO,-0.4716,technosphere,False
3896,"transport, freight, lorry >32 metric ton, EURO5",lorry_32,ton kilometer,RER,"thermochemical recycling, MP",ChemRecMP_export,GLO,0.0120,technosphere,False
3897,Water,09872080-d143-4fb1-a3a5-647b077107ff,cubic meter,"('air', 'non-urban air or from high stacks')","thermochemical recycling, MP",ChemRecMP_export,GLO,0.0199,biosphere,False


#### 2.2.2 Add parameters

In [1767]:
# Excel sheet name (EWU-Dashboard)
params_sheet_name='Parameter'

In [1768]:
scenarios=['default']

for s in range(n_scenarios):
    scenarios.append('S'+str(s+1))

In [1769]:
param_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Parameters')
param_df

,Parameter-ID,Parameter Description,Default Amount,Unit,Group
0,P1,Transport of LWP Waste to Sorting,9.500000e-03,ton kilometer,NaN
1,P2,Collection of LWP Waste,1.200000e-01,ton kilometer,NaN
2,P3,"MBT, electricity consumption",6.186000e-02,kilowatt hour,NaN
3,P4,"MBT, diesel consumption",8.048250e-02,megajoule,NaN
4,P5,"MBT, facility construction",2.500000e-10,unit,NaN
...,...,...,...,...,...
346,P347,waste collection - TP2 (TP_DRS_SingleUse),1.705263e-01,-,4.0
347,P348,waste collection - TP3 (TP_LWP),3.692416e-01,-,4.0
348,P349,waste collection - TP4 (TP_RW_Inc),1.913953e-01,-,4.0
349,P350,waste collection - TP5 (TP_RW_MBT),3.837097e-02,-,4.0


In [1770]:
parameter_df=pd.DataFrame()
p=1

parameter_df.loc[0,'Parameter-ID']='P0'
parameter_df.loc[0,'Parameter']='Waste Quantity'
for s in range(n_scenarios):
    parameter_df.loc[0,'Parameter Value - S'+str(s+1)]=WQ_kg
parameter_df.loc[0,'Parameter Value - default']=WQ_kg
parameter_df.loc[0,'Unit']='kilogram'

if general_info['Type of parameterization']=='predefined parameters':
    param_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Parameters')
    for i in param_df.index:
        parameter_df.loc[i+1,'Parameter-ID']=param_df.loc[i,'Parameter-ID']
        parameter_df.loc[i+1,'Parameter']=param_df.loc[i,'Parameter Description']
        parameter_df.loc[i+1,'Unit']=param_df.loc[i,'Unit']  
        parameter_df.loc[i+1,'Group']=param_df.loc[i,'Group']  
        for s in range(n_scenarios):
            parameter_df.loc[i+1,'Parameter Value - S'+str(s+1)]=param_df.loc[i,'Default Amount']  
        parameter_df.loc[i+1,'Parameter Value - default']=param_df.loc[i,'Default Amount']
        
    param_exchanges_df=exchanges_df[pd.isnull(exchanges_df['formula'])==False]
    for param_ind in param_exchanges_df.index:
        exc_ind=LCI_df[(LCI_df['input code']==param_exchanges_df.loc[param_ind, 'input code'])&
                       (LCI_df['activity code']==param_exchanges_df.loc[param_ind, 'activity code'])].index[0]
        LCI_df.loc[exc_ind, 'formula']=param_exchanges_df.loc[param_ind, 'formula']
    

if general_info['Type of parameterization']=='automatically generated parameters':
    if general_info['Parameterize exchanges']=='technosphere':
        for i in LCI_df.index:
            if LCI_df.loc[i,'type']=='technosphere':
                param='P'+str(p+1)
                LCI_df.loc[i,'formula']=param
                parameter_df.loc[p,'Parameter-ID']=param
                parameter_df.loc[p,'Parameter']='FROM: '+LCI_df.loc[i,'input']+' ('+LCI_df.loc[i,'input location']+', '+LCI_df.loc[i,'input unit']+'), TO: '+LCI_df.loc[i,'activity']+'('+LCI_df.loc[i,'activity location']+')'
                parameter_df.loc[p,'Unit']=LCI_df.loc[i,'input unit']       
                for s in range(n_scenarios):
                    parameter_df.loc[p,'Parameter Value - S'+str(s+1)]=LCI_df.loc[i,'amount']
                parameter_df.loc[p,'Parameter Value - default']=LCI_df.loc[i,'amount']
                if LCI_df.loc[i,'material flow'] == True:
                    parameter_df.loc[p, 'Group']=foreground_system_groups[LCI_df.loc[i,'activity code']]
                p=p+1
                
    if general_info['Parameterize exchanges']=='biosphere':
        for i in LCI_df.index:
            if LCI_df.loc[i,'type']=='biosphere':
                param='P'+str(p+1)
                LCI_df.loc[i,'formula']=param
                parameter_df.loc[p,'Parameter-ID']=param
                parameter_df.loc[p,'Parameter']='FROM: '+LCI_df.loc[i,'input']+' ('+LCI_df.loc[i,'input location']+', '+LCI_df.loc[i,'input unit']+'), TO: '+LCI_df.loc[i,'activity']+'('+LCI_df.loc[i,'activity location']+')'
                parameter_df.loc[p,'Unit']=LCI_df.loc[i,'input unit']       
                for s in range(n_scenarios):
                    parameter_df.loc[p,'Parameter Value - S'+str(s+1)]=LCI_df.loc[i,'amount']
                parameter_df.loc[p,'Parameter Value - default']=LCI_df.loc[i,'amount']
                if LCI_df.loc[i,'material flow'] == True:
                    parameter_df.loc[p, 'Group']=foreground_system_groups[LCI_df.loc[i,'activity code']]
                p=p+1
                
    if general_info['Parameterize exchanges']=='technosphere & biosphere':
        for i in LCI_df.index:
            if LCI_df.loc[i,'type']!='production':
                param='P'+str(p+1)
                LCI_df.loc[i,'formula']=param
                parameter_df.loc[p,'Parameter-ID']=param
                parameter_df.loc[p,'Parameter']='FROM: '+LCI_df.loc[i,'input']+' ('+LCI_df.loc[i,'input location']+', '+LCI_df.loc[i,'input unit']+'), TO: '+LCI_df.loc[i,'activity']+'('+LCI_df.loc[i,'activity location']+')'
                parameter_df.loc[p,'Unit']=LCI_df.loc[i,'input unit']       
                for s in range(n_scenarios):
                    parameter_df.loc[p,'Parameter Value - S'+str(s+1)]=LCI_df.loc[i,'amount']
                parameter_df.loc[p,'Parameter Value - default']=LCI_df.loc[i,'amount']
                if LCI_df.loc[i,'material flow'] == True:
                    parameter_df.loc[p, 'Group']=foreground_system_groups[LCI_df.loc[i,'activity code']]
                p=p+1

In [1771]:
params_col_dict={}
for c,col in enumerate(parameter_df.columns):
    params_col_dict[col]=excel_cols[c]
    
params_row_dict={}
for p,param in enumerate(parameter_df['Parameter-ID']):
    params_row_dict[param]=p+2
#params_row_dict

In [1772]:
LCI_df[LCI_df['activity code']=='Waste_Generation']

,input,input code,input unit,input location,activity,activity code,activity location,amount,type,material flow,formula
2234,waste generation,Waste_Generation,kilogram,DE,waste generation,Waste_Generation,DE,1.000000,production,False,NaN
2235,deposit-refund-system for reusable bottles,TP_DRS_Reuse,kilogram,DE,waste generation,Waste_Generation,DE,0.225466,technosphere,True,P346
2236,deposit-refund-system for single-use bottles,TP_DRS_SingleUse,kilogram,DE,waste generation,Waste_Generation,DE,0.170526,technosphere,True,P347
2237,treatment of lightweight packaging waste (mech...,TP_LWP,kilogram,DE,waste generation,Waste_Generation,DE,0.369242,technosphere,True,P348
2238,"treatment of residual waste, MSWI",TP_RW_Inc,kilogram,DE,waste generation,Waste_Generation,DE,0.191395,technosphere,True,P349
2239,"treatment of residual waste, MBT",TP_RW_MBT,kilogram,DE,waste generation,Waste_Generation,DE,0.038371,technosphere,True,P350
2240,littering of plastic packaging waste,TP_Littering,kilogram,DE,waste generation,Waste_Generation,DE,0.005000,technosphere,True,P351


#### 2.2.3 LCA calculations for unique exchanges

**Technosphere**

In [1854]:
LCI_df_tech=LCI_df[LCI_df['type']=='technosphere']
unique_inputs_tech=LCI_df_tech['input code'].unique()
unique_activities_tech=LCI_df_tech['activity code'].unique()
CF_dict_techno={}

for d,db in enumerate(dbs):
    functional_units_tech=[]
    reference_actcodes_tech=[]

    for actcode in unique_inputs_tech:
        if actcode not in unique_activities_tech:
            reference_actcodes_tech.append(actcode)
            functional_unit={db.get(actcode):1}
            functional_units_tech.append(functional_unit)
        
    calculation_setup = {'inv': functional_units_tech, 'ia': LCIA_methods}
    bw.calculation_setups['excel tool'] = calculation_setup
    mlca = bw.MultiLCA('excel tool')
    
    CF_dict_techno_db={}

    for a,actcode in enumerate(reference_actcodes_tech):
        CF_dict_techno_flow={}
        for i,IC in enumerate(LCIA_method_names):
            CF_dict_techno_flow[IC]=mlca.results[a,i]
        CF_dict_techno_db[actcode]=CF_dict_techno_flow
    
    CF_dict_techno[db_names[d]]=CF_dict_techno_db

In [1855]:
CF_techno_dfs={}
for db in db_names:
    CF_techno_df_db=pd.DataFrame.from_dict(CF_dict_techno[db]).transpose()
    CF_techno_dfs[db]=CF_techno_df_db

In [1856]:
CF_techno_sheet_names={}

for db in db_names:
    CF_techno_sheet_names[db]='CF_techno_'+db

**Biosphere**

In [109]:
LCIA_fp=r"{}".format(general_info['LCIA implementation file path'])

In [110]:
LCIA=pd.read_excel(LCIA_fp,sheet_name='CFs')

LCI_df_bio=LCI_df[LCI_df['type']=='biosphere']
unique_inputs_bio=LCI_df_bio['input code'].unique()
unique_activities_bio=LCI_df_bio['activity code'].unique()

functional_units_bio=[]
reference_actcodes_bio=[]

for actcode in unique_inputs_bio:
    if actcode not in unique_activities_bio:
        reference_actcodes_bio.append(actcode)
        functional_unit={biosphere.get(actcode):1}
        functional_units_bio.append(functional_unit)
        
functional_units=[*functional_units_tech,*functional_units_bio]
reference_actcodes=[*reference_actcodes_tech,*reference_actcodes_bio]

CF_dfs={}
for i,IC in enumerate(LCIA_method_names):
    CF_dfs[IC]=LCIA[(LCIA['Method']==LCIA_methods[i][0])&(LCIA['Category']==LCIA_methods[i][1])&(LCIA['Indicator']==LCIA_methods[i][2])]
    
    
CF_dict_bio={}
for actcode in reference_actcodes_bio:
    bioflow=biosphere.get(actcode)
    bioflow_name=bioflow['name']
    bioflow_c0=bioflow['categories'][0]
    if len(bioflow['categories'])==2:
        bioflow_c1=bioflow['categories'][1]
    else:
        bioflow_c1='unspecified'
    CF_dict_bio_bioflow={}
    for IC in LCIA_method_names:
        CF_df=CF_dfs[IC]
        CF_df_filtered=CF_df[(CF_df['Name']==bioflow_name)&(CF_df['Compartment']==bioflow_c0)&(CF_df['Subcompartment']==bioflow_c1)]
        if len(CF_df_filtered)==0:
            CF=0
        else:
            CF=CF_df_filtered['CF'].values[0]
        CF_dict_bio_bioflow[IC]=CF
    CF_dict_bio[actcode]=CF_dict_bio_bioflow    

In [471]:
CF_bio_df=pd.DataFrame.from_dict(CF_dict_bio).transpose()

In [476]:
CF_bio_sheet_name='CF_bio'

#### 2.2.4 Add characterization factors to LCI-DataFrame

In [1857]:
LCI_df_sheet_names={}

for db in db_names:
    for s in scenarios:
        LCI_df_sheet_names[db+'_'+s]='LCI_'+db+'_'+s

In [1859]:
LCI_dbs={}

for d,db in enumerate(dbs):
    for s in scenarios:
        LCI_df_db=LCI_df.copy()

        for i in LCI_df_db.index:
            input_code=LCI_df_db.loc[i,'input code']
            exc_type=LCI_df_db.loc[i,'type']
            if exc_type=='biosphere':
                for IC in LCIA_method_names:
                    LCI_df_db.loc[i,IC]=CF_dict_bio[input_code][IC]
            if exc_type=='technosphere':
                if input_code in reference_actcodes_tech:
                    for IC in LCIA_method_names:
                        LCI_df_db.loc[i,IC]=CF_dict_techno[db_names[d]][input_code][IC] 
                        
        LCI_df_db['formula excel']=LCI_df_db['formula']
        parameters=parameter_df['Parameter-ID']        
            
        # Excel formula
        for i in LCI_df_db.index:
            formula=LCI_df_db.loc[i,'formula']
            if type(formula)==str:
                formula_new=formula
                for p,param in enumerate(reversed(parameters)):
                    formula_new=formula_new.replace(param,params_sheet_name+'!'\
                                +params_col_dict['Parameter Value - '+s]+str(params_row_dict[param]))
                formula_new='='+formula_new
                formula_new=formula_new.replace('amount',excel_cols[LCI_df_db.columns.get_loc('amount')]+str(i+2))
                LCI_df_db.loc[i,'formula excel']=formula_new
            else:
                LCI_df_db.loc[i,'formula excel']='='+excel_cols[LCI_df_db.columns.get_loc('amount')]\
                                                    +str(i+2)
        
        for i in LCI_df_db.index:
            if LCI_df_db.loc[i,'type']!='production':
                for IC in LCIA_method_names:
                    LCI_df_db.loc[i,IC+' Impact']='='+excel_cols[LCI_df_db.columns.get_loc('formula excel')]\
                                                        +str(i+2)+'*'\
                                                    +excel_cols[LCI_df_db.columns.get_loc(IC)]+str(i+2)
                    
        for actcode in LCI_df_db['activity code'].unique():
            sub_df=LCI_df_db[LCI_df_db['activity code']==actcode]
            for n,IC in enumerate(LCIA_method_names):
                formula='='
                col=excel_cols[LCI_df_db.columns.get_loc(IC+' Impact')]
                for i in sub_df.index[1:]:
                    ind=str(i+2)
                    formula=formula+'+'+col+ind
                LCI_df_db.loc[sub_df.index[0], IC+' Impact']=formula
                
        LCI_df_db_prod=LCI_df_db[LCI_df_db['type']=='production']
        prod_dict={}
        
        for i in LCI_df_db_prod.index:
            actcode=LCI_df_db_prod.loc[i,'activity code']
            prod_dict[actcode]=i+2
            
        for i in LCI_df_db.index:
            if LCI_df_db.loc[i,'type']!='production':
                input_code=LCI_df_db.loc[i,'input code']
                if input_code in prod_dict.keys():
                    for n,IC in enumerate(LCIA_method_names):
                        col=excel_cols[LCI_df_db.columns.get_loc(IC+' Impact')]
                        LCI_df_db.loc[i,IC]='='+col+str(prod_dict[input_code])+'*'\
                                               +excel_cols[LCI_df_db.columns.get_loc('formula excel')]\
                                               +str(prod_dict[input_code])
                    
        LCI_dbs[db_names[d]+'_'+s]=LCI_df_db

#### 2.2.5 LCA Results

In [1860]:
LCA_results_sheet_names={}

for db in db_names:
    for s in scenarios:
        LCA_results_sheet_names[db+'_'+s]='LCA_'+db+'_'+s

In [1861]:
LCA_results={}

for db in db_names:
    for s in scenarios:
        LCA_results_db=pd.DataFrame()
        actcodes=act_codes
        actnames=act_names

        for a,actcode in enumerate(actcodes):
            LCA_results_db.loc[a,'Pathway']=actnames[a]
            for i,IC in enumerate(LCIA_method_names):
                col=IC+' Impact'
                LCA_results_db.loc[a,IC]='=LCI_'+db+'_'+s+'!'+LCI_df_col_dict[col]+str(prod_dict[actcode])\
                                         +'/LCIA_Methods!D'+str(i+2)

        LCA_results[db+'_'+s]=LCA_results_db

### 2.3 Perturbation Analysis

In [ ]:
if general_info['Conduct perturbation analysis'] == 'yes':

    default_values={}
    for i in parameter_df.index:
        default_values[parameter_df.loc[i,'Parameter-ID']]=parameter_df.loc[i,'Parameter Value - default']

    perturbation_runs={}
    for r in parameter_df.index:
        run='run'+str(r+1)
        run_params={}
        for i in parameter_df.index:
            if i!=r:
                run_params[parameter_df.loc[i,'Parameter-ID']]=parameter_df.loc[i,'Parameter Value - default']
            else:
                run_params[parameter_df.loc[i,'Parameter-ID']]=parameter_df.loc[i,'Parameter Value - default']*1.001
        perturbation_runs[run]=run_params

    relevant_indices_runs={}
    for run in perturbation_runs.keys():
        relevant_indices={}
        for i in LCI_df_db.index:
            formula=LCI_df_db.loc[i,'formula']
            if type(formula)==str:
                evaluated_formula=formula
                for p,param in enumerate(reversed(perturbation_runs[run].keys())):
                    evaluated_formula=evaluated_formula.replace(param,str(perturbation_runs[run][param]))
                evaluated_formula=evaluated_formula.replace('amount', str(LCI_df_db.loc[i,'amount']))
                evaluated_formula=float(eval(evaluated_formula))
                if LCI_df_db.loc[i,'amount']!=evaluated_formula:
                    relevant_indices[i]=evaluated_formula
                    #print(run,i,evaluated_formula)
        relevant_indices_runs[run]=relevant_indices
    relevant_indices_runs

    exc_dict={}
    for i in LCI_df.index:
        act=eidb.get(LCI_df.loc[i,'activity code'])
        exchg=[exc for exc in act.exchanges() if exc.input.as_dict()['code'] ==LCI_df.loc[i,'input code']][0]
        exc_dict[i]=exchg
    exc_dict

    perturbation_df=pd.DataFrame()

    wg=eidb.get(WasteGen_code)
    
    non_stochastic_lca=bw.LCA({wg:1})
    non_stochastic_lca.lci()
    C_matrices={}
    for i,method in enumerate(LCIA_methods):
        non_stochastic_lca.switch_method(method)
        C_matrices[LCIA_method_names[i]] = non_stochastic_lca.characterization_matrix
    
    for r,run in enumerate(perturbation_runs.keys()):
        print(run)
        if run == 'run1':
            lca = bw.LCA({wg:1.001})   

        else:
            for ind in relevant_indices_runs[run].keys():
                default_amount=LCI_df.loc[ind,'amount']
                exc_dict[ind]['amount']=relevant_indices_runs[run][ind]
                exc_dict[ind].save()

            lca = bw.LCA({wg:1})          

        lca.lci()
        for IC in LCIA_method_names:
            perturbation_df.loc['P'+str(r),IC]=(C_matrices[IC]*lca.inventory).sum()

        if run != 'run1':
            for ind in relevant_indices_runs[run].keys():
                default_amount=LCI_df.loc[ind,'amount']
                exc_dict[ind]['amount']=default_amount
                exc_dict[ind].save()        

    # calculation of sensitivity ratios
    calculation_setup = {'inv': [{wg:1}], 'ia': LCIA_methods}
    bw.calculation_setups['perturbation'] = calculation_setup
    default_mlca = bw.MultiLCA('perturbation')       

    senstivity_ratio_df=pd.DataFrame()

    for i,ind in enumerate(perturbation_df.index):
        for c,col in enumerate(perturbation_df.columns):
            delta_rel_lca=(perturbation_df.loc[ind,col]-default_mlca.results[0][c])/default_mlca.results[0][c]
            delta_rel_param=0.001
            senstivity_ratio_df.loc[ind,col]=delta_rel_lca/delta_rel_param

    senstivity_ratio_df=senstivity_ratio_df.reset_index(drop=True)

    parameter_df=pd.concat([parameter_df, senstivity_ratio_df], axis=1)

### 2.4 Waste Composition

In [1862]:
Waste_Comp_sheet_names={}

for s in scenarios:
    Waste_Comp_sheet_names[s]='Waste_Composition_'+s

In [1863]:
Waste_Comp={}

for s in scenarios:
    Waste_Comp_S=pd.DataFrame()
    for a,actname in enumerate(Pathway_names):
        sub_df=LCI_df[LCI_df['activity code']==Pathway_codes[a]]
        for i in sub_df.index:
            for mat in material_codes:
                if sub_df.loc[i,'input code'] in activity_material_dict.keys():
                    if mat == activity_material_dict[sub_df.loc[i,'input code']]:
                        Waste_Comp_S.loc[mat,actname]='=ABS(LCI_default_'+s+'!'+LCI_df_col_dict['formula excel']+str(i+2)+')'    
    Waste_Comp_S=Waste_Comp_S.loc[material_codes]
    Waste_Comp[s]=Waste_Comp_S

In [1864]:
Waste_Comp_col_dict={}
for c,col in enumerate(Waste_Comp_S.columns):
    Waste_Comp_col_dict[col]=excel_cols[c+1]

### 2.5 Waste Generation

In [1865]:
Waste_Gen_sheet_names={}

for s in scenarios:
    Waste_Gen_sheet_names[s]='Waste_Generation_'+s

In [1866]:
Waste_Gen={}

for s in scenarios:
    Waste_Gen_S=pd.DataFrame(index=act_names)
    name_code_dict={}
    for a,ac in enumerate(Pathway_codes):
        name_code_dict[Pathway_names[a]]=ac

    sub_df=LCI_df[LCI_df['activity code']==WasteGen_code]
    for i in sub_df.index:
        for a,ac in enumerate(Pathway_codes):
            if ac == sub_df.loc[i,'input code']:
                Waste_Gen_S.loc[Pathway_names[a],'relative']='=ABS(LCI_default_'+s+'!'\
                                                        +LCI_df_col_dict['formula excel']+str(i+2)+')'
                Waste_Gen_S.loc[Pathway_names[a],'absolute [kg]']='=ABS(LCI_default_'+s+'!'\
                                                +LCI_df_col_dict['formula excel']+str(i+2)+'*'\
                                                +"HLOOKUP(\"Parameter Value - "+s+"\",Parameter!C1:"\
                                                +excel_cols[n_scenarios+2]+"2,2,FALSE))"
    #Waste_Gen_S=Waste_Gen_S.sort_index()
    for c,col in enumerate(Waste_Gen_S.columns):
        Waste_Gen_S.loc[act_names[0],col]='=SUM('+excel_cols[c+1]+str(3)+':'+excel_cols[c+1]+str(len(Waste_Gen_S)+1)+')'
    
    Waste_Gen[s]=Waste_Gen_S

### 2.6 Original Environmental Value

#### 2.6.1 Materials

In [1867]:
EV_df=pd.read_excel("ExcelTool_GeneratorInput_Template.xlsx", index_col=0, sheet_name='EnvironmentalValue')

In [1868]:
OEV_material_sheet_names={}

for db in db_names:
    OEV_material_sheet_names[db]='OEV_materials_'+db

In [1869]:
material_dict={}

for d,db in enumerate(dbs):
    material_dict_db={}
    for i in EV_df.index:
        material_dict_db[i]=[act for act in db if act['name']==EV_df.loc[i,'Material activity name']
                            and act['unit']==EV_df.loc[i,'Material activity unit']
                            and act['location']==EV_df.loc[i,'Material activity location']][0]
    material_dict[db_names[d]]=material_dict_db

In [1870]:
OEV_materials={}

for d,db in enumerate(db_names):
    functional_units=[]
    for act in material_dict[db].values():
        functional_units.append({act:1})
    functional_units

    calculation_setup = {'inv': functional_units, 'ia': LCIA_methods}

    bw.calculation_setups['materials'] = calculation_setup

    mlca = bw.MultiLCA('materials')

    OEV_material_norm=np.empty(np.shape(mlca.results))

    for f,fu in enumerate(functional_units):
        OEV_material_norm[f]=mlca.results[f]/nfs

    OEV_materials_db=pd.DataFrame(OEV_material_norm, 
                 index=EV_df.index,
                 columns=LCIA_method_names)

    OEV_materials[db]=OEV_materials_db

In [1871]:
OEV_materials_col_dict={}
for c,col in enumerate(OEV_materials[db].columns):
    OEV_materials_col_dict[col]=excel_cols[c+1]

In [1872]:
mlca.results

array([[1.97792544e+00, 4.91781548e-08, 3.25268797e-08, 1.02751037e-07,
        7.17047266e-08, 6.99884854e-02, 7.13389338e-03, 7.06421123e-03,
        1.51802825e-02, 2.33186557e-05, 1.36746997e-03, 1.35405533e+00,
        7.35044379e+00, 9.47085250e-01, 1.58025632e-05, 7.27571645e+01],
       [2.02434107e+00, 3.52039608e-08, 2.59761299e-08, 9.93455667e-08,
        7.17521662e-08, 5.51838939e-02, 6.66606820e-03, 7.17039853e-03,
        1.48730395e-02, 2.54176544e-05, 1.34020336e-03, 1.28108165e+00,
        6.68071136e+00, 8.21130810e-01, 1.53583450e-05, 7.46171794e+01],
       [2.19940125e+00, 1.49382242e-05, 6.61718909e-08, 2.11363212e-07,
        9.12718921e-08, 1.43637043e-01, 7.29701344e-03, 7.73327344e-03,
        1.71349824e-02, 3.10895470e-05, 1.61452646e-03, 2.70456970e+00,
        1.48103435e+01, 1.41120337e+00, 3.76865480e-04, 6.55024156e+01],
       [3.92968696e+00, 1.39063037e-08, 2.77679006e-08, 3.82112944e-08,
        1.33447952e-07, 5.08360253e-03, 9.99274609e-03, 1.395

#### 2.6.2 Pathways

In [1873]:
OEV_Pathways_sheet_names={}

for db in db_names:
    for s in scenarios:
        OEV_Pathways_sheet_names[db+'_'+s]='OEV_Pathways_'+db+'_'+s

In [1874]:
OEV_Pathways={}

for db in db_names:
    for s in scenarios:
        OEV_Pathways_db=pd.DataFrame(index=act_names)
        actnames=Pathway_names

        for Pathway in actnames:
            for n,IC in enumerate(LCIA_method_names):
                OEV_Pathways_db.loc[Pathway,IC]='=SUMPRODUCT('+OEV_material_sheet_names[db]+'!'+OEV_materials_col_dict[IC]+'2:'+\
                                    OEV_material_sheet_names[db]+'!'+OEV_materials_col_dict[IC]+str(len(EV_df.index)+1)+','+ \
                                    Waste_Comp_sheet_names[s]+'!'+Waste_Comp_col_dict[Pathway]+'2:'+\
                                    Waste_Comp_sheet_names[s]+'!'+Waste_Comp_col_dict[Pathway]+str(len(EV_df.index)+1) \
                                    +')'
    
        for i,IC in enumerate(LCIA_method_names):
            OEV_Pathways_db.loc[act_names[0],IC]='=SUMPRODUCT('+Waste_Gen_sheet_names[s]+'!B3:'+Waste_Gen_sheet_names[s]+'!B'+str(len(actnames)+2)+','\
                                            +excel_cols[i+1]+str(3)+':'+excel_cols[i+1]+str(len(actnames)+2)+')'

        OEV_Pathways[db+'_'+s]=OEV_Pathways_db

### 2.7 Environmental Waste Utilization

#### 2.7.1 Per Pathway

In [1875]:
Indicator_sheet_names={}

for db in db_names:
    for s in scenarios:
        Indicator_sheet_names[db+'_'+s]='Indicator_'+db+'_'+s

In [1876]:
Indicator={}

for db in db_names:
    for s in scenarios:
        Indicator_db=pd.DataFrame(index=OEV_Pathways_db.index,columns=OEV_Pathways_db.columns)

        for i,ind in enumerate(Indicator_db.index):
            for c,col in enumerate(Indicator_db.columns):
                Indicator_db.loc[ind,col]='=-'+LCA_results_sheet_names[db+'_'+s]+'!'\
                +excel_cols[c+1]+str(i+2)+'/'+OEV_Pathways_sheet_names[db+'_'+s]+'!'+excel_cols[c+1]+str(i+2)
        
        Indicator[db+'_'+s]=Indicator_db

#### 2.7.2 Aggregated

In [1877]:
Weighted_Indicator_sheet_names={}

for db in db_names:
    for s in scenarios:
        Weighted_Indicator_sheet_names[db+'_'+s]='W_Ind_'+db+'_'+s

In [1878]:
Weighted_Indicator={}

for db in db_names:
    for s in scenarios:
        Weighted_Indicator_db=pd.DataFrame()

        for a,act in enumerate(Indicator_db.index):
            for w,wm in enumerate(Weighting.columns):
                formula='='
                for i,IC in enumerate(LCIA_method_names):
                    formula=formula+'+'+Indicator_sheet_names[db+'_'+s]+'!'+excel_cols[i+1]+str(a+2)\
                            +'*'+Weighting_sheet_name+'!'+excel_cols[w+1]+str(i+2)
                Weighted_Indicator_db.loc[act,wm]=formula
        Weighted_Indicator[db+'_'+s]=Weighted_Indicator_db

### 2.8 Supply Chain

In [1879]:
supply_chain_sheet_names={}

for s in scenarios:
    for act in act_names:
        supply_chain_sheet_names[s+'_'+act]='supply_chain_'+s+'_'+act

In [1880]:
def supply_chain_per_Pathway(LCI,
                       activity_df,
                       supply_chain_df,
                       db, scenario, n_scenarios, 
                       amount_col=None,
                       excel_cols=excel_cols,
                       previous_input_code=None, 
                       previous_input_name=None,
                       first=True, x=None):
    if first == True:
        x=1
        amount_col=excel_cols[LCI.columns.get_loc('formula excel')]
        reference_amount="=VLOOKUP(\""+previous_input_name+"\",Waste_Generation_"+scenario+"!A1:C100,3,FALSE)*" \
                            +str(LCI[(LCI['type']=='production')&(LCI['input code']==previous_input_code)]['amount'].iloc[0])
        supply_chain_df.loc[previous_input_code,'amount']=reference_amount
        
    if previous_input_code not in LCI['activity code'].unique():
        return
    else:
        if x ==1000:
            return
        else:
            x=x+1
            ref=LCI[LCI['activity code']==previous_input_code]
            previous_index=supply_chain_df.index.get_loc(previous_input_code)+2
            production_index=ref[(ref['input code']==previous_input_code)&(ref['type']=='production')].index[0]+2
            for i in ref.index:
                if ref.loc[i,'type']!='production':
                    input_code=ref.loc[i,'input code']
                    if len(LCI[(LCI['input code']==input_code)&(LCI['type']=='production')])>0:
                        act_index=LCI[(LCI['input code']==input_code)&(LCI['type']=='production')].index[0]+2
                    else:
                        act_index=LCI[(LCI['input code']==input_code)].index[0]+2
                    if pd.isnull(supply_chain_df.loc[input_code,'amount'])==True:
                        supply_chain_df.loc[input_code,'amount']='=B'+str(previous_index)+'*'+'LCI_'+db+'_'+scenario+'!'\
                                                                +amount_col+str(i+2)+'*'+'LCI_'+db+'_'+scenario+'!'\
                                                                +amount_col+str(production_index)
                    else:
                        formula='B'+str(previous_index)+'*'+'LCI_'+db+'_'+scenario+'!'+amount_col+str(i+2)+'*'\
                                                                +'LCI_'+db+'_'+scenario+'!'\
                                                                +amount_col+str(production_index)
                        if formula not in supply_chain_df.loc[input_code,'amount']:
                            supply_chain_df.loc[input_code,'amount']=supply_chain_df.loc[input_code,'amount']+'+'+formula
                    previous_input_code=input_code
                    supply_chain_per_Pathway(LCI=LCI,
                                       activity_df=activity_df,
                                       supply_chain_df=supply_chain_df,
                                       db=db, scenario=scenario, n_scenarios=n_scenarios, 
                                       amount_col=amount_col,
                                       excel_cols=excel_cols,
                                       previous_input_code=previous_input_code, 
                                       first=False,x=x)

In [1881]:
LCI=LCI_dbs[db_names[0]+'_'+scenarios[0]]

In [1882]:
supply_chain={}
db=db_names[0]

for scenario in scenarios:
    for a,actcode in enumerate(act_codes):
        #print(scenario+'_'+act_names[a], actcode)
        supply_chain_df=pd.DataFrame(index=LCI['input code'].unique())

        supply_chain_per_Pathway(LCI,
                       activity_df, 
                       supply_chain_df,
                       db, scenario, n_scenarios,
                       previous_input_code=actcode,
                       previous_input_name=act_names[a])

        supply_chain_df=supply_chain_df.replace(np.nan,0)

        supply_chain[scenario+'_'+act_names[a]]=supply_chain_df

### 2.9 Contribution Analysis

#### 2.9.1 Pathways

In [1883]:
CA_Pathways_sheet_names={}

for db in db_names:
    for s in scenarios:
        CA_Pathways_sheet_names[db+'_'+s]='CA_Pathways_'+db+'_'+s

In [1884]:
CA_Pathways={}
amount_col=excel_cols[LCI_df_db.columns.get_loc('formula excel')]

for db in db_names:
    for s in scenarios:
        CA_Pathways_db=pd.DataFrame(index=OEV_Pathways_db.index, columns=LCIA_method_names)

        for n,IC in enumerate(CA_Pathways_db.columns):
            for i,ind in enumerate(CA_Pathways_db.index):
                 CA_Pathways_db.loc[ind,IC]='='+LCA_results_sheet_names[db+'_'+s]+'!'+excel_cols[n+1]\
                                    +str(i+2)+'*'+Waste_Gen_sheet_names[s]+'!C'+str(i+2)

        CA_Pathways[db+'_'+s]=CA_Pathways_db

#### 2.9.2 Activities

In [1885]:
CA_acts_sheet_names={}

for db in db_names:
    for s in scenarios:
        CA_acts_sheet_names[db+'_'+s]='CA_acts_'+db+'_'+s

In [1886]:
# activity tags
activity_tag_dict={}

for tag in activity_df['Activity tag'].unique():
    if pd.isnull(tag)==False:
        tag_list=[]
        sub_df=activity_df[activity_df['Activity tag']==tag]
        for i in sub_df.index:
            sub_dict={}
            sub_dict['name']=activity_df.loc[i,'Activity name']
            sub_dict['code']=activity_df.loc[i,'Activity code']
            sub_dict['unit']=activity_df.loc[i,'Unit']
            sub_dict['location']=activity_df.loc[i,'Location']
            tag_list.append(sub_dict)
        activity_tag_dict[tag]=tag_list

In [1887]:
df=LCI_dbs['default_default']
db='default'
scenario='default'

In [1888]:
for tag in activity_tag_dict.keys():
    for tag_act in activity_tag_dict[tag]:
        name=tag_act['name']
        location=tag_act['location']
        unit=tag_act['unit']
        code=tag_act['code']
        for i in df.index:
            if (df.loc[i,'input']==name)&(df.loc[i,'input location']==location)&(df.loc[i,'input unit']==unit)&(df.loc[i,'input code']==code):
                df.loc[i,'activity tag']=tag

In [1889]:
for db in db_names:
    for scenario in scenarios:
        for tag in activity_tag_dict.keys():
            for tag_act in activity_tag_dict[tag]:
                if len(df[(df['activity']==tag_act['name'])&(df['type']=='production')])>0:
                    foreground=True
                    for i in df[(df['activity code']==tag_act['code'])&(df['type']=='production')].index:
                        sub_dict={}
                        sub=df[(df['activity code']==tag_act['code'])&(df['activity location']==tag_act['location'])]
                        index=sub[(sub['activity tag'].isnull()==False)&(sub['type']=='production')].index[0]
                        sub_tags=sub[(sub['activity tag'].isnull()==False)&(sub['type']!='production')].index
                        tag_act['sub_tags']=sub_tags
                        ref_amount= LCI_df_col_dict['formula excel']+str(index+2)       
                else:
                    index=df[(df['input']==tag_act['name'])].index[0]
                    ref_amount= LCI_df_col_dict['formula excel']+str(index+2)
                    foreground=False
                tag_act['index']=index
                tag_act['ref_amount']=ref_amount
                code=df.loc[index,'input code']
                tag_act['code']=code
                tag_act['foreground']=foreground

In [1890]:
CA_acts={}

for db in db_names:
    for scenario in scenarios:
        CA_acts_df=pd.DataFrame()
        for tag in activity_tag_dict:
            for i,IC in enumerate(LCIA_method_names):
                unit_impact='('
                
                for t,tag_act in enumerate(activity_tag_dict[tag]):
                    if t != 0:
                        unit_impact=unit_impact+'+'
                        
                    if tag_act['foreground']==True:
                        unit_impact=unit_impact+supply_chain_sheet_names[scenario+'_'+act_names[0]]+'!B'\
                                        +str(supply_chain_df.index.get_loc(tag_act['code'])+2)+'*'+'('+'LCI_'+db+'_'+scenario+'!'+LCI_df_col_dict[IC+' Impact']\
                                        +str(tag_act['index']+2)
                        if 'sub_tags' in tag_act.keys():
                            for sub_tag in tag_act['sub_tags']:
                                unit_impact=unit_impact+'-LCI_'+db+'_'+scenario+'!'+LCI_df_col_dict[IC+' Impact']\
                                            +str(sub_tag+2)
                        unit_impact=unit_impact+')'+'/'+'LCI_'+db+'_'+scenario+'!'+tag_act['ref_amount']
                    else:
                        formula=supply_chain_sheet_names[scenario+'_'+act_names[0]]+'!B'\
                                        +str(supply_chain_df.index.get_loc(tag_act['code'])+2)+'*'+'LCI_'+db+'_'+scenario+'!'+LCI_df_col_dict[IC]\
                                        +str(tag_act['index']+2)
                        if formula not in unit_impact:
                            unit_impact=unit_impact+formula
                        else:
                            unit_impact=unit_impact[:-1]

                CA_acts_df.loc[tag,IC]='='+unit_impact+')'+'/LCIA_Methods!D'+str(i+2)
        CA_acts[db+'_'+scenario]=CA_acts_df

In [1891]:
n_tags=len(activity_tag_dict.keys())

In [1892]:
for db in db_names:
    for scenario in scenarios:
        CA_acts_df=CA_acts[db+'_'+scenario]
        for i,IC in enumerate(LCIA_method_names):
            col=excel_cols[i+1]
            reference_amount='ABS('+supply_chain_sheet_names[scenario+'_'+act_names[0]]+'!B'\
                                    +str(supply_chain_df.index.get_loc(WasteGen_code)+2)+')'
            CA_acts_df.loc['Others', IC]='=LCA_'+db+'_'+scenario+'!'+col+'2*'+reference_amount+'-SUM('+col+'2:'\
                                        +col+str(n_tags+1)+')'

### 2.10 EWU-Components: Treatment - Utilization - Material

#### 2.10.1 Per Impact Category

In [1893]:
TUM_sheet_names={}

for db in db_names:
    for s in scenarios:
        for TUM_type in ['Treatm', 'Util', 'Mat']:
            TUM_sheet_names[db+'_'+s+'_'+TUM_type]='TUM_'+db+'_'+s+'_'+TUM_type

In [1894]:
sc_indices={}
LCI_indices={}
for wuc in waste_utilization_codes:
    sc_indices[wuc]=str(supply_chain_df.index.get_loc(wuc)+2)
    LCI_indices[wuc]=str(LCI[LCI['input code']==wuc].index[0]+2)

In [1895]:
TUM={}

for db in db_names:
    for s in scenarios:
        for TUM_type in ['Treatm', 'Util', 'Mat']:
            TUM_df=pd.DataFrame(index=act_names, columns=LCIA_method_names)
            if TUM_type == 'Util':
                for act in TUM_df.index:
                    for i,IC in enumerate(LCIA_method_names):
                        formula='=('
                        for wuc in waste_utilization_codes:
                            formula = formula + '+(supply_chain_'+s+'_'+act+'!B'+sc_indices[wuc]+'*'+'LCI_'+db+'_'+scenario+'!'+LCI_df_col_dict[IC]\
                                                    +LCI_indices[wuc]+')'                   
                        TUM_df.loc[act, IC]=formula+')/LCIA_Methods!D'+str(i+2)
                TUM[db+'_'+s+'_'+'Util']=TUM_df
            if TUM_type == 'Treatm':
                for a,act in enumerate(TUM_df.index):
                    for i,IC in enumerate(LCIA_method_names):
                        TUM_df.loc[act, IC]='=CA_Pathways_'+db+'_'+s+'!'+excel_cols[i+1]+str(a+2)+\
                                            '-TUM_'+db+'_'+s+'_'+'Util'+'!'+excel_cols[i+1]+str(a+2)
                TUM[db+'_'+s+'_'+'Treatm']=TUM_df
            if TUM_type == 'Mat':
                for a,act in enumerate(TUM_df.index):
                    for i,IC in enumerate(LCIA_method_names):
                        TUM_df.loc[act, IC]='=OEV_Pathways_'+db+'_'+s+'!'+excel_cols[i+1]+str(a+2)+"*VLOOKUP(\""+act+"\",Waste_Generation_"+scenario+"!A1:C100,3,FALSE)"
                TUM[db+'_'+s+'_'+'Mat']=TUM_df                

#### 2.10.2 Aggregated

In [1896]:
Weighted_TUM_sheet_names={}

for db in db_names:
    for s in scenarios:
        for TUM_type in ['Treatm', 'Util', 'Mat']:
            Weighted_TUM_sheet_names[db+'_'+s+'_'+TUM_type]='WTUM_'+db+'_'+s+'_'+TUM_type

In [1897]:
Weighted_TUM={}

for db in db_names:
    for s in scenarios:
        for TUM_type in ['Treatm', 'Util', 'Mat']:
            Weighted_TUM_df=pd.DataFrame()

            for a,act in enumerate(TUM_df.index):
                for w,wm in enumerate(Weighting.columns):
                    formula='='
                    for i,IC in enumerate(LCIA_method_names):
                        formula=formula+'+'+TUM_sheet_names[db+'_'+s+'_'+TUM_type]+'!'+excel_cols[i+1]+str(a+2)\
                                +'*'+Weighting_sheet_name+'!'+excel_cols[w+1]+str(i+2)
                    Weighted_TUM_df.loc[act,wm]=formula
            Weighted_TUM[db+'_'+s+'_'+TUM_type]=Weighted_TUM_df

### 2.11 Dashboard

#### 2.11.1 Figure 1: Indicator Scores per Impact Category (Spider Chart)

In [1898]:
figure1_sheet_name='Figure1_Data'

In [1899]:
figure1_index='=EWU_Dashboard!B3' #selected treatment path / system
figure1_df=pd.DataFrame(columns=LCIA_method_names)
for i,IC in enumerate(LCIA_method_names):
    for a,act in enumerate(act_names):
        figure1_df.loc[act,IC]="=IF(EWU_Dashboard!B3=A"+str(a+2)+",INDIRECT(\"\'\"&"+\
                                            excel_cols[i+1]+str(len(act_names)+2)+\
                                            "&\"\'!"+excel_cols[i+1]\
                                            +"\"&VLOOKUP(EWU_Dashboard!B3,dict!A2:B100,2,FALSE)),#N/A)"
    figure1_df.loc['sheet_name',IC]="=\"Indicator_\"&"+excel_cols[i+1]+str(len(act_names)+3)\
                                        +"&\"_\"&EWU_Dashboard!B5"
    figure1_df.loc['db_short',IC]="=VLOOKUP(EWU_Dashboard!B4,dict!A2:B100,2,FALSE)"

#### 2.11.2 Figure 2: Aggregated Indicator Scores (Bar Chart)

In [1900]:
figure2_sheet_name='Figure2_Data'

In [1901]:
figure2_index='=EWU_Dashboard!B6' #selected weighting method
figure2_df=pd.DataFrame(columns=act_names)

for a,act in enumerate(act_names):
    figure2_df.loc[figure2_index,act]="=HLOOKUP(A2,INDIRECT("+excel_cols[a+1]+"3"+"&\"!B1:"\
                                        +excel_cols[len(Weighting.columns)]+str(len(act_names)+1)+"\"),"+\
                                        "VLOOKUP("+excel_cols[a+1]+"1,dict!A2:B100,2,FALSE),FALSE)"
    figure2_df.loc['sheet_name',act]="=\"W_Ind_\"&"+excel_cols[a+1]+"4&\"_\"&EWU_Dashboard!B5"
    figure2_df.loc['db_short',act]="=VLOOKUP(EWU_Dashboard!B4,dict!A2:B100,2,FALSE)"

#### 2.11.3 Figure 3: Contribution Analysis - Pathways (Bar Chart - Stacked)

In [1902]:
figure3_sheet_name='Figure3_Data'

In [1903]:
figure3_df=pd.DataFrame(columns=LCIA_method_names)
fig3_sheet_name="=\"CA_Pathways_\"&B2&\"_\"&EWU_Dashboard!B5" #cell B1
fig3_db_short="=VLOOKUP(EWU_Dashboard!B4,dict!A2:B100,2,FALSE)" # cell B2

for a,act in enumerate(act_names):
    for i,IC in enumerate(LCIA_method_names):
        figure3_df.loc[act,IC]="=INDIRECT(B1&\"!"+excel_cols[i+1]+str(a+2)+"\")"#+"*INDIRECT(B3&\"!C"+str(a+2)+"\")"

#### 2.11.4 Figure 4: Contribution Analysis - Processes (Bar Chart - Stacked)

In [1904]:
figure4_sheet_name='Figure4_Data'

In [1905]:
figure4_df=pd.DataFrame(columns=LCIA_method_names)
fig4_sheet_name="=\"CA_acts_\"&B2&\"_\"&EWU_Dashboard!B5" #cell B1
fig4_db_short="=VLOOKUP(EWU_Dashboard!B4,dict!A2:B100,2,FALSE)" # cell B2

for a,act in enumerate(CA_acts_df.index):
    for i,IC in enumerate(LCIA_method_names):
        figure4_df.loc[act,IC]="=INDIRECT(B1&\"!"+excel_cols[i+1]+str(a+2)+"\")"

#### 2.11.5 Figure 5: Scenario Analysis - Foreground System (Bar Chart)

In [1906]:
figure5_sheet_name='Figure5_Data'

In [1907]:
n_weightingsets=len(Weighting.columns)
n_acts=len(act_names)

In [1908]:
figure5_index='=EWU_Dashboard!B3' #selected treatment path / system
figure5_df=pd.DataFrame(columns=scenarios)

for s,scenario in enumerate(scenarios):
    for a,act in enumerate(act_names):
        figure5_df.loc[act,scenario]="=IF(EWU_Dashboard!B3=A"+str(a+2)+",HLOOKUP(EWU_Dashboard!B6,INDIRECT("+excel_cols[s+1]\
                                                +str(len(act_names)+2)+"&\"!A1:"\
                                                +excel_cols[n_weightingsets]+str(n_acts+1)\
                                                +"\"),VLOOKUP(EWU_Dashboard!B3,dict!A2:B100,2,FALSE),FALSE),0)"
    figure5_df.loc['sheet_name',scenario]="=\"W_Ind_\"&"+excel_cols[s+1]+str(len(act_names)+3)+"&\"_\"&\""+scenario+"\""
    figure5_df.loc['db_short',scenario]="=VLOOKUP(EWU_Dashboard!B4,dict!A2:B100,2,FALSE)"

#### 2.11.6 Figure 6: Scenario Analysis - Background System (Bar Chart)

In [1909]:
figure6_sheet_name='Figure6_Data'

In [1910]:
figure6_index='=EWU_Dashboard!B3' #selected treatment path / system
figure6_df=pd.DataFrame(columns=db_names)

for d,db in enumerate(db_names):
    for a,act in enumerate(act_names):
        figure6_df.loc[act,db]="=IF(EWU_Dashboard!B3=A"+str(a+2)+",HLOOKUP("+excel_cols[d+1]+str(len(act_names)+3)+",INDIRECT("+excel_cols[d+1]\
                                            +str(len(act_names)+4)+"&\"!A1:"\
                                            +excel_cols[n_weightingsets]+str(n_acts+1)\
                                            +"\"),VLOOKUP(EWU_Dashboard!B3,dict!A1:B100,2,FALSE),FALSE),0)"
    figure6_df.loc['scenario',db]="=EWU_Dashboard!B5"
    figure6_df.loc['weighting_method',db]="=EWU_Dashboard!B6"
    figure6_df.loc['sheet_name',db]="=\"W_Ind_\"&"+excel_cols[d+1]+"1&\"_\"&"+excel_cols[d+1]+str(len(act_names)+2)

#### 2.11.7 Figure 7: EWU - Components

In [1911]:
figure7_sheet_name='Figure7_Data'

In [1912]:
figure7_df=pd.DataFrame(columns=['Treatment', 'Utilization', 'Material'])
col_shorts=['Treatm', 'Util', 'Mat']

for c,col in enumerate(figure7_df.columns):
    for a,act in enumerate(act_names):
        figure7_df.loc[act,col]="=HLOOKUP("+excel_cols[c+1]+str(len(act_names)+4)+",INDIRECT("+excel_cols[c+1]\
                                            +str(len(act_names)+5)+"&\"!A1:"\
                                            +excel_cols[n_weightingsets]+str(n_acts+1)\
                                            +"\"),"+str(a+2)+",FALSE)"
    figure7_df.loc['db_short',col]="=VLOOKUP(EWU_Dashboard!B4,dict!A2:B100,2,FALSE)"
    figure7_df.loc['scenario',col]="=EWU_Dashboard!B5"
    figure7_df.loc['weighting_method',col]="=EWU_Dashboard!B6"
    figure7_df.loc['sheet_name',col]="=\"WTUM_\"&"+excel_cols[c+1]+str(len(act_names)+2)+"&\"_\"&"+\
                                        excel_cols[c+1]+str(len(act_names)+3)+"&\"_"+col_shorts[c]+"\""

## 3. Export to Excel

### 3.1 Format

In [1913]:
EWU_dashboard='EWU_Dashboard.xlsx'

In [1914]:
writer = pd.ExcelWriter(EWU_dashboard, engine='xlsxwriter')
workbook = writer.book

In [1915]:
title_format = workbook.add_format({'num_format': '0.00','text_wrap':False, 
                                         'font':'Arial', 'font_size':22,'bold':True, 
                                         'border':0,'border_color':'black'})
subtitle_format = workbook.add_format({'num_format': '0.00','text_wrap':False, 
                                         'font':'Arial', 'font_size':22,'bold':True, 
                                         'border':0,'border_color':'black'})
text_format = workbook.add_format({'num_format': '0%','text_wrap':False, 
                                         'font':'Arial', 'font_size':18, 
                                         'border':1,'border_color':'black'})
text_format2 = workbook.add_format({'num_format': '0%','text_wrap':False, 
                                         'font':'Arial', 'font_size':18, 
                                         'bold': True, 'font_color': 'white',
                                         'fg_color':'#595959',
                                         'border':1,'border_color':'black'})
heading_format = workbook.add_format({'num_format': '0.00','text_wrap':True, 
                                         'font':'Arial', 'font_size':10,'bold':True, 
                                         'border':1,'border_color':'black'})
percentage_format = workbook.add_format({'num_format': '0%','text_wrap':True, 
                                         'font':'Arial', 'font_size':10, 
                                         'border':1,'border_color':'black'})
number_format = workbook.add_format({'num_format': '0.00','text_wrap':True, 
                                         'font':'Arial', 'font_size':10, 
                                         'border':1,'border_color':'black'})
scientific_format = workbook.add_format({'num_format': '0.00E+00','text_wrap':True, 
                                         'font':'Arial', 'font_size':10, 
                                         'border':1,'border_color':'black'})
background_format = workbook.add_format({'bg_color':'white'})
int_format = workbook.add_format({'num_format': '0','text_wrap':True, 
                                         'font':'Arial', 'font_size':10, 
                                         'border':1,'border_color':'black'})

### 3.2 Colors

In [1916]:
color_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Format', index_col=0)

In [1917]:
if len(color_df)==0:
    color_dict={}
else:
    color_dict=color_df.iloc[:,0].to_dict()
    
color_keys=[*act_names, *activity_tags]

In [1918]:
ck_notincl=[]
for ck in color_keys:
    if ck not in color_dict.keys():
        ck_notincl.append(ck)
        
add_colors=mcp.gen_color(cmap="turbo",n=len(ck_notincl))

for c,ck in enumerate(ck_notincl):
    color_dict[ck]=add_colors[c]

### 3.3 Create workbooks

In [1919]:
Dashboard_sheet=workbook.add_worksheet('EWU_Dashboard')
params_sheet=workbook.add_worksheet(params_sheet_name)

figure1_sheet=workbook.add_worksheet(figure1_sheet_name)
figure2_sheet=workbook.add_worksheet(figure2_sheet_name)
figure3_sheet=workbook.add_worksheet(figure3_sheet_name)
figure4_sheet=workbook.add_worksheet(figure4_sheet_name)
figure5_sheet=workbook.add_worksheet(figure5_sheet_name)
figure6_sheet=workbook.add_worksheet(figure6_sheet_name)
figure7_sheet=workbook.add_worksheet(figure7_sheet_name)
documentation_sheet=workbook.add_worksheet('Documentation')
dict_sheet=workbook.add_worksheet('dict')

Indicator_sheets={}
for db_s in Indicator_sheet_names.keys():
    Indicator_sheets[db_s]=workbook.add_worksheet(Indicator_sheet_names[db_s])
    
Weighted_Indicator_sheets={}
for db_s in Weighted_Indicator_sheet_names.keys():
    Weighted_Indicator_sheets[db_s]=workbook.add_worksheet(Weighted_Indicator_sheet_names[db_s])
    
Weighting_sheet=workbook.add_worksheet(Weighting_sheet_name)

LCA_results_sheets={}
for db_s in LCA_results_sheet_names.keys():
    LCA_results_sheets[db_s]=workbook.add_worksheet(LCA_results_sheet_names[db_s])

CA_Pathways_sheets={}
for db_s in CA_Pathways_sheet_names.keys():
    CA_Pathways_sheets[db_s]=workbook.add_worksheet(CA_Pathways_sheet_names[db_s])

CA_acts_sheets={}
for db_s in CA_acts_sheet_names.keys():
    CA_acts_sheets[db_s]=workbook.add_worksheet(CA_acts_sheet_names[db_s])

Waste_Gen_sheets={}
for db_s in Waste_Gen_sheet_names.keys():
    Waste_Gen_sheets[db_s]=workbook.add_worksheet(Waste_Gen_sheet_names[db_s])
    
Waste_Comp_sheets={}
for db_s in Waste_Comp_sheet_names.keys():
    Waste_Comp_sheets[db_s]=workbook.add_worksheet(Waste_Comp_sheet_names[db_s])
    
LCI_df_sheets={}
for db_s in LCI_df_sheet_names.keys():    
    LCI_df_sheets[db_s]=workbook.add_worksheet(LCI_df_sheet_names[db_s])
    
LCIAmethod_sheet=workbook.add_worksheet(LCIAmethod_sheet_name)

OEV_material_sheets={}
for db_s in OEV_material_sheet_names.keys():
    OEV_material_sheets[db_s]=workbook.add_worksheet(OEV_material_sheet_names[db_s])
    
OEV_Pathways_sheets={}
for db_s in OEV_Pathways_sheet_names.keys():
    OEV_Pathways_sheets[db_s]=workbook.add_worksheet(OEV_Pathways_sheet_names[db_s])
    
supply_chain_sheets={}
for db_s in supply_chain_sheet_names.keys():
    supply_chain_sheets[db_s]=workbook.add_worksheet(supply_chain_sheet_names[db_s])
    
TUM_sheets={}
for db_s_m in TUM_sheet_names.keys():
    TUM_sheets[db_s_m]=workbook.add_worksheet(TUM_sheet_names[db_s_m])
    
Weighted_TUM_sheets={}
for db_s_m in Weighted_TUM_sheet_names.keys():
    Weighted_TUM_sheets[db_s_m]=workbook.add_worksheet(Weighted_TUM_sheet_names[db_s_m])

### 3.4 Write data to workbooks

#### 3.4.1 Indicator

In [1920]:
# Indicator
for db in db_names:
    for s in scenarios:
        Indicator_sheet=Indicator_sheets[db+'_'+s]
        for c,col in enumerate(Indicator[db+'_'+s].columns):
            Indicator_sheet.write(0,c+1,Indicator[db+'_'+s].columns[c],heading_format)
            for i,ind in enumerate(Indicator[db+'_'+s].index):
                if c==0:
                    Indicator_sheet.write(i+1,0,Indicator[db+'_'+s].index[i],heading_format)
                Indicator_sheet.write(i+1,c+1,Indicator[db+'_'+s].loc[ind,col],percentage_format)
        Indicator_sheet.set_tab_color('green')
        Indicator_sheet.hide()

#### 3.4.2 Weighted Indicator

In [1921]:
# Weighted Indicator
for db in db_names:
    for s in scenarios:
        Weighted_Indicator_sheet=Weighted_Indicator_sheets[db+'_'+s]
        for c,col in enumerate(Weighted_Indicator[db+'_'+s].columns):
            Weighted_Indicator_sheet.write(0,c+1,Weighted_Indicator[db+'_'+s].columns[c],heading_format)    
            for i,ind in enumerate(Weighted_Indicator[db+'_'+s].index):
                if c==0:
                    Weighted_Indicator_sheet.write(i+1,0,Weighted_Indicator[db+'_'+s].index[i],heading_format)
                Weighted_Indicator_sheet.write(i+1,c+1,Weighted_Indicator[db+'_'+s].loc[ind,col],percentage_format)
        Weighted_Indicator_sheet.set_column_pixels(0,c+1, 100)    
        Weighted_Indicator_sheet.set_tab_color('green')
        Weighted_Indicator_sheet.hide()

#### 3.4.3 Weighting Methods

In [1922]:
# Weighting
for c,col in enumerate(Weighting.columns):
    Weighting_sheet.write(0,c+1,Weighting.columns[c],heading_format)    
    for i,ind in enumerate(Weighting.index):
        if c==0:
            Weighting_sheet.write(i+1,0,Weighting.index[i],heading_format)
        Weighting_sheet.write(i+1,c+1,Weighting.loc[ind,col],percentage_format)
Weighting_sheet.set_column_pixels(0,len(Weighting.columns)+1, 100)   
Weighting_sheet.hide()

#### 3.4.4 LCA Results

In [1923]:
# LCA Results
for db in db_names:
    for s in scenarios:
        LCA_results_sheet=LCA_results_sheets[db+'_'+s]
        for c,col in enumerate(LCA_results[db+'_'+s].columns):
            if c>0:
                LCA_results_sheet.write(0,c,LCA_results[db+'_'+s].columns[c],heading_format)
                for i,ind in enumerate(LCA_results[db+'_'+s].index):
                    if c==1:
                        LCA_results_sheet.write(i+1,0,LCA_results[db+'_'+s].loc[ind,'Pathway'],heading_format)
                    LCA_results_sheet.write(i+1,c,LCA_results[db+'_'+s].loc[ind,col],scientific_format)
        LCA_results_sheet.hide()

#### 3.4.5 Contribution Analysis - Pathways

In [1924]:
# CA_Pathways
for db in db_names:
    for s in scenarios:
        CA_Pathways_sheet=CA_Pathways_sheets[db+'_'+s]
        for c,col in enumerate(CA_Pathways[db+'_'+s].columns):
            CA_Pathways_sheet.write(0,c+1,CA_Pathways[db+'_'+s].columns[c],heading_format)
            for i,ind in enumerate(CA_Pathways[db+'_'+s].index):
                if c==0:
                    CA_Pathways_sheet.write(i+1,0,CA_Pathways[db+'_'+s].index[i],heading_format)
                CA_Pathways_sheet.write(i+1,c+1,CA_Pathways[db+'_'+s].loc[ind,col],scientific_format)
        CA_Pathways_sheet.hide()

#### 3.4.6 Contribution Analysis - Activities

In [1925]:
# CA_acts
for db in db_names:
    for s in scenarios:
        CA_acts_sheet=CA_acts_sheets[db+'_'+s]
        for c,col in enumerate(CA_acts[db+'_'+s].columns):
            CA_acts_sheet.write(0,c+1,CA_acts[db+'_'+s].columns[c],heading_format)
            for i,ind in enumerate(CA_acts[db+'_'+s].index):
                if c==0:
                    CA_acts_sheet.write(i+1,0,CA_acts[db+'_'+s].index[i],heading_format)
                CA_acts_sheet.write(i+1,c+1,CA_acts[db+'_'+s].loc[ind,col],scientific_format)
        CA_acts_sheet.hide()

#### 3.4.7 Supply Chain

In [1926]:
def split_formulas(x):
    ints=[i for i in range(1,int(len(x)/7000)+2) if i < (len(x)/7000+1)]

    sections=[]
    for c in ints:
        lower_bound=7000*c
        if c != ints[-1]:
            upper_bound=7000*c+1000
            x_section=x[lower_bound:upper_bound]
            plus_position=x_section.rfind('+')+lower_bound
            if c==1:
                lower_plus_bound=0
            plus_section=x[lower_plus_bound:plus_position]
            if c != 1:
                plus_section='='+plus_section
        else:
            upper_bound=len(x)+1
            plus_section=x[lower_plus_bound:]
            plus_section='='+plus_section
        sections.append(plus_section)
        lower_plus_bound=plus_position+1
        
    return sections

In [1927]:
# supply_chain
for s in scenarios:
    for actname in act_names:
        supply_chain_sheet=supply_chain_sheets[s+'_'+actname]
        for c,col in enumerate(supply_chain[s+'_'+actname].columns):
            supply_chain_sheet.write(0,c+1,supply_chain[s+'_'+actname].columns[c],heading_format)
            for i,ind in enumerate(supply_chain[s+'_'+actname].index):
                if c==0:
                    supply_chain_sheet.write(i+1,0,supply_chain[s+'_'+actname].index[i],heading_format)
                supply_chain_sheet.write(i+1,c+1,supply_chain[s+'_'+actname].loc[ind,col],scientific_format)
        supply_chain_sheet.hide()

In [1928]:
# # supply_chain
# for s in scenarios:
#     for actname in act_names:
#         supply_chain_sheet=supply_chain_sheets[s+'_'+actname]
#         for c,col in enumerate(supply_chain[s+'_'+actname].columns):
#             supply_chain_sheet.write(0,c+1,supply_chain[s+'_'+actname].columns[c],heading_format)
#             for i,ind in enumerate(supply_chain[s+'_'+actname].index):
#                 if c==0:
#                     supply_chain_sheet.write(i+1,0,supply_chain[s+'_'+actname].index[i],heading_format)
#                 if type(supply_chain[s+'_'+actname].loc[ind,col])==str:
#                     if len(supply_chain[s+'_'+actname].loc[ind,col]) < 8100:
#                         supply_chain_sheet.write(i+1,c+1,supply_chain[s+'_'+actname].loc[ind,col],scientific_format)
#                     else:
#                         sections=split_formulas(supply_chain[s+'_'+actname].loc[ind,col])
#                         sumformula='='
#                         for sec,section in enumerate(sections):
#                             supply_chain_sheet.write(i+1,c+2+sec,section,scientific_format)
#                             sumformula=sumformula+'+'+excel_cols[c+2+sec]+str(i+2)
#                         supply_chain_sheet.write(i+1,c+1,sumformula,scientific_format)
#                 else:
#                     supply_chain_sheet.write(i+1,c+1,supply_chain[s+'_'+actname].loc[ind,col],scientific_format)
                        
#         supply_chain_sheet.hide()

#### 3.4.8 EWU-Components: Treatment - Utilization - Material

In [1929]:
# TUM
for db in db_names:
    for s in scenarios:
        for TUM_type in ['Treatm', 'Util', 'Mat']:
            TUM_sheet=TUM_sheets[db+'_'+s+'_'+TUM_type]
            for c,col in enumerate(TUM[db+'_'+s+'_'+TUM_type].columns):
                TUM_sheet.write(0,c+1,TUM[db+'_'+s+'_'+TUM_type].columns[c],heading_format)
                for i,ind in enumerate(TUM[db+'_'+s+'_'+TUM_type].index):
                    if c==0:
                        TUM_sheet.write(i+1,0,TUM[db+'_'+s+'_'+TUM_type].index[i],heading_format)
                    TUM_sheet.write(i+1,c+1,TUM[db+'_'+s+'_'+TUM_type].loc[ind,col],scientific_format)
                    TUM_sheet.hide()

#### 3.4.9 Weighted EWU-Components: Treatment - Utilization - Material

In [1930]:
# TUM
for db in db_names:
    for s in scenarios:
        for TUM_type in ['Treatm', 'Util', 'Mat']:
            Weighted_TUM_sheet=Weighted_TUM_sheets[db+'_'+s+'_'+TUM_type]
            for c,col in enumerate(Weighted_TUM[db+'_'+s+'_'+TUM_type].columns):
                Weighted_TUM_sheet.write(0,c+1,Weighted_TUM[db+'_'+s+'_'+TUM_type].columns[c],heading_format)
                for i,ind in enumerate(Weighted_TUM[db+'_'+s+'_'+TUM_type].index):
                    if c==0:
                        Weighted_TUM_sheet.write(i+1,0,Weighted_TUM[db+'_'+s+'_'+TUM_type].index[i],heading_format)
                    Weighted_TUM_sheet.write(i+1,c+1,Weighted_TUM[db+'_'+s+'_'+TUM_type].loc[ind,col],scientific_format)
                    Weighted_TUM_sheet.hide()

#### 3.4.10 Parameters

In [1931]:
# Parameters
for c,col in enumerate(parameter_df.columns):
    if c>0:
        params_sheet.write(0,c,parameter_df.columns[c],heading_format)
        for i,ind in enumerate(parameter_df.index):
            if c==1:
                params_sheet.write(i+1,0,str(parameter_df.loc[ind,'Parameter-ID']),heading_format)
            if col == 'Group':
                if pd.isnull(parameter_df.loc[ind,col])==False:
                    params_sheet.write(i+1,c,parameter_df.loc[ind,col],int_format)
                else:
                    params_sheet.write(i+1,c,'None',int_format)
            else:
                try:
                    params_sheet.write(i+1,c,parameter_df.loc[ind,col],scientific_format)
                except:
                    pass
params_sheet.set_column_pixels(1,len(parameter_df.columns)+1, 100)  

sc_format = workbook.add_format({'bold': 1, 'bg_color': 'yellow'})
col_default = excel_cols[parameter_df.columns.get_loc("Parameter Value - "+scenarios[0])]
scenario_cols=[col for col in parameter_df.columns if ('Parameter Value' in col) and ('Parameter Value - default' not in col)]
for i in parameter_df.index:
    for col in scenario_cols:
        col_name = excel_cols[parameter_df.columns.get_loc(col)]
        cell=col_name+str(i+2)
        params_sheet.conditional_format(cell, 
                                        {'type': 'cell',
                                                 'criteria': "not equal to",
                                                 'value': col_default+str(i+2),
                                                 'format': sc_format})
    
if general_info['Conduct perturbation analysis'] == 'yes':
    col_x = parameter_df.columns.get_loc(LCIA_method_names[0])
    #col_y = parameter_df.columns.get_loc(LCIA_method_names[-1]) + 1
    for i,IC in enumerate(LCIA_method_names):
        min_val=parameter_df[IC].min()
        max_val=parameter_df[IC].max()
        cells=excel_cols[col_x+i]+str(2)+':'+excel_cols[col_x+i]+str(len(parameter_df)+2)
        params_sheet.conditional_format(cells, {'type': '3_color_scale',
                                             'min_color': "#C00000",
                                             'mid_color': "#FFFFFF",
                                             'max_color': "#9BBB59",
                                             'min_value':min_val,
                                             'mid_value':0,
                                             'max_value':max_val})

#params_sheet.set_tab_color('yellow')

#### 3.4.11 Waste Quantitities

In [1932]:
# Waste Generation
for s in scenarios:
    Waste_Gen_sheet=Waste_Gen_sheets[s]
    for c,col in enumerate(Waste_Gen[s].columns):
        Waste_Gen_sheet.write(0,c+1,Waste_Gen[s].columns[c],heading_format)    
        for i,ind in enumerate(Waste_Gen[s].index):
            if c==0:
                Waste_Gen_sheet.write(i+1,0,Waste_Gen[s].index[i],heading_format)
                Waste_Gen_sheet.write(i+1,c+1,Waste_Gen[s].loc[ind,col],percentage_format)
            if c==1:
                Waste_Gen_sheet.write(i+1,c+1,Waste_Gen[s].loc[ind,col],number_format)
    Waste_Gen_sheet.set_column_pixels(0,len(Waste_Gen[s].columns)+1, 100)  
    Waste_Gen_sheet.hide()

#### 3.4.12 Waste Composition

In [1933]:
# Waste Composition
for s in scenarios:
    Waste_Comp_sheet=Waste_Comp_sheets[s]
    for c,col in enumerate(Waste_Comp[s].columns):
        Waste_Comp_sheet.write(0,c+1,Waste_Comp[s].columns[c],heading_format)    
        for i,ind in enumerate(Waste_Comp[s].index):
            if c==0:
                Waste_Comp_sheet.write(i+1,0,Waste_Comp[s].index[i],heading_format)
            try:
                Waste_Comp_sheet.write(i+1,c+1,Waste_Comp[s].loc[ind,col],percentage_format)
            except:
                pass
    Waste_Comp_sheet.set_column_pixels(0,len(Waste_Comp[s].columns)+1, 100)
    Waste_Comp_sheet.hide()

#### 3.4.13 LCI

In [1934]:
# LCI
for db in db_names:
    for s in scenarios:
        LCI_df_sheet=LCI_df_sheets[db+'_'+s]
        for c,col in enumerate(LCI_dbs[db+'_'+s].columns):
            LCI_df_sheet.write(0,c,LCI_dbs[db+'_'+s].columns[c],heading_format)
            for i,ind in enumerate(LCI_dbs[db+'_'+s].index):
                try:
                    LCI_df_sheet.write(i+1,c,LCI_dbs[db+'_'+s].loc[ind,col],number_format)
                except:
                    pass
        LCI_df_sheet.set_column_pixels(1,len(LCI_dbs[db+'_'+s].columns)+1, 100)
        LCI_df_sheet.hide()

#### 3.4.14 Original Environmental Value (Material Value)

In [1935]:
# Original Environmental Value - materials
for db in db_names:
    OEV_material_sheet=OEV_material_sheets[db]
    for c,col in enumerate(OEV_materials[db].columns):
        OEV_material_sheet.write(0,c+1,OEV_materials[db].columns[c],heading_format)    
        for i,ind in enumerate(OEV_materials[db].index):
            if c==0:
                OEV_material_sheet.write(i+1,0,OEV_materials[db].index[i],heading_format)
            try:
                OEV_material_sheet.write(i+1,c+1,OEV_materials[db].loc[ind,col],scientific_format)
            except:
                pass
    OEV_material_sheet.set_column_pixels(0,len(OEV_materials[db].columns)+1, 100)
    OEV_material_sheet.hide()

In [1936]:
# Original Environmental Value - Treatment Paths
for db in db_names:
    for s in scenarios:
        OEV_Pathways_sheet=OEV_Pathways_sheets[db+'_'+s]
        for c,col in enumerate(OEV_Pathways[db+'_'+s].columns):
            OEV_Pathways_sheet.write(0,c+1,OEV_Pathways[db+'_'+s].columns[c],heading_format)    
            for i,ind in enumerate(OEV_Pathways[db+'_'+s].index):
                if c==0:
                    OEV_Pathways_sheet.write(i+1,0,OEV_Pathways[db+'_'+s].index[i],heading_format)
                try:
                    OEV_Pathways_sheet.write(i+1,c+1,OEV_Pathways[db+'_'+s].loc[ind,col],scientific_format)
                except:
                    pass
        OEV_Pathways_sheet.set_column_pixels(0,len(OEV_Pathways[db+'_'+s].columns)+1, 100)  
        OEV_Pathways_sheet.hide()

#### 3.4.15 Normalization Factors

In [1937]:
# NormalizationFactor
for c,col in enumerate(LCIAmethod_df.columns):
    LCIAmethod_sheet.write(0,c+1,LCIAmethod_df.columns[c],heading_format)
    for i,ind in enumerate(LCIAmethod_df.index):
        if c==0:
            LCIAmethod_sheet.write(i+1,0,LCIAmethod_df.index[i],heading_format)
        LCIAmethod_sheet.write(i+1,c+1,LCIAmethod_df.loc[ind,col],scientific_format)
        
LCIAmethod_sheet.set_column_pixels(1,1, 300)
LCIAmethod_sheet.set_column_pixels(3,3, 100)
LCIAmethod_sheet.hide()

#### 3.4.16 Dictionary

In [1938]:
dict_df=pd.DataFrame(columns=['long','short'])

In [1939]:
i=0
for a,act in enumerate(Indicator_db.index):
    dict_df.loc[i,'long']=act
    dict_df.loc[i,'short']=i+2
    i=i+1
    
for d,db in enumerate(db_names):
    dict_df.loc[i,'long']=dbs[d].name
    dict_df.loc[i,'short']=db
    i=i+1

In [1940]:
# dictionary
for c,col in enumerate(dict_df.columns):
    dict_sheet.write(0,c,dict_df.columns[c],heading_format)
    for i,ind in enumerate(dict_df.index):
        dict_sheet.write(i+1,c,dict_df.loc[ind,col])
        
dict_sheet.set_column_pixels(1,1, 300)
dict_sheet.set_column_pixels(3,3, 100)
dict_sheet.hide()

#### 3.4.17 Documentation

In [1941]:
#Get locally imported modules from current notebook
#htPathways://stackoverflow.com/questions/40428931/package-for-listing-version-of-packages-used-in-a-jupyter-notebook
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

In [1942]:
activity_references=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', 
                                   sheet_name='Activities')[['Activity code', 
                                                            'Activity name',
                                                           'Unit',
                                                           'Location','Reference']]

In [1943]:
package_df=pd.DataFrame()
for p,package in enumerate(requirements):
    package_df.loc[p,'package']=package[0]
    package_df.loc[p,'version']=package[1]

In [1944]:
activity_references=activity_references.replace(np.nan, 'Assumption')

In [1945]:
premise_doc=premise_scenarios.dropna(axis=1,how='all').dropna()
premise_doc=premise_doc.reset_index(drop=True)
premise_doc['Update']=[str([i for i in premise_update.index]) for i in range(len(premise_doc))]

In [1946]:
for c,col in enumerate(activity_references.columns):
    documentation_sheet.write(0,c,activity_references.columns[c],heading_format)
    for i,ind in enumerate(activity_references.index):
        documentation_sheet.write(i+1,c,activity_references.loc[ind,col],percentage_format)
        
first_row=len(activity_references)+5

for c,col in enumerate(package_df.columns):
    documentation_sheet.write(first_row,c,package_df.columns[c],heading_format)
    for i,ind in enumerate(package_df.index):
        documentation_sheet.write(i+1+first_row,c,package_df.loc[ind,col],percentage_format)
        
documentation_sheet.write(first_row+len(package_df)+2,0,'Database',percentage_format)
documentation_sheet.write(first_row+len(package_df)+2,1,general_info['Database name'],percentage_format)

documentation_sheet.write(first_row+len(package_df)+4,0,'Background system scenarios',heading_format)
for c,col in enumerate(premise_doc.columns):
    documentation_sheet.write(first_row+len(package_df)+5,c,str(premise_doc.columns[c]),heading_format)
    for i,ind in enumerate(premise_doc.index):
        documentation_sheet.write(i+1+first_row+len(package_df)+5,c,premise_doc.loc[ind,col],percentage_format)

documentation_sheet.set_column_pixels(0,4, 300)
documentation_sheet.set_column_pixels(2,3, 100)

documentation_sheet.hide_gridlines(option=2)
documentation_sheet.hide()

#### 3.4.18 Figure Data

In [1947]:
# Figure1
for c,col in enumerate(figure1_df.columns):
    figure1_sheet.write(0,c+1,col,heading_format)
    for i,ind in enumerate(figure1_df.index):
        if c==0:
            figure1_sheet.write(i+1,0,figure1_df.index[i],heading_format)
        cell=excel_cols[c+1]+str(i+2)
        figure1_sheet.write_dynamic_array_formula(cell,formula=figure1_df.loc[ind,col], cell_format=percentage_format)
figure1_sheet.set_tab_color('orange')
figure1_sheet.hide()

In [1948]:
# Figure2
for c,col in enumerate(figure2_df.columns):
    figure2_sheet.write(0,c+1,col,heading_format)
    for i,ind in enumerate(figure2_df.index):
        if c==0:
            figure2_sheet.write(i+1,0,figure2_df.index[i],heading_format)
        cell=excel_cols[c+1]+str(i+2)
        figure2_sheet.write_dynamic_array_formula(cell,formula=figure2_df.loc[ind,col], cell_format=percentage_format)
figure2_sheet.set_tab_color('orange')
figure2_sheet.hide()

In [1949]:
# Figure3
figure3_sheet.write('A1','sheet_name',heading_format)
figure3_sheet.write('A2','db_short',heading_format)
#figure3_sheet.write('A3','wg_sheet_name',heading_format)
figure3_sheet.write('B1',fig3_sheet_name,heading_format)
figure3_sheet.write('B2',fig3_db_short,heading_format)
#figure3_sheet.write('B3',fig3_wg_sheet_name,heading_format)

for c,col in enumerate(figure3_df.columns):
    figure3_sheet.write(4,c+1,col,heading_format)
    for i,ind in enumerate(figure3_df.index):
        if c==0:
            figure3_sheet.write(i+5,0,figure3_df.index[i],heading_format)
        cell=excel_cols[c+1]+str(i+6)
        figure3_sheet.write_dynamic_array_formula(cell,formula=figure3_df.loc[ind,col], cell_format=scientific_format)
figure3_sheet.set_tab_color('orange')
figure3_sheet.hide()

In [1950]:
# Figure4
figure4_sheet.write('A1','sheet_name',heading_format)
figure4_sheet.write('A2','db_short',heading_format)
figure4_sheet.write('B1',fig4_sheet_name,heading_format)
figure4_sheet.write('B2',fig4_db_short,heading_format)

for c,col in enumerate(figure4_df.columns):
    figure4_sheet.write(4,c+1,col,heading_format)
    for i,ind in enumerate(figure4_df.index):
        if c==0:
            figure4_sheet.write(i+5,0,figure4_df.index[i],heading_format)
        cell=excel_cols[c+1]+str(i+6)
        figure4_sheet.write_dynamic_array_formula(cell,formula=figure4_df.loc[ind,col], cell_format=scientific_format)
figure4_sheet.set_tab_color('orange')
figure4_sheet.hide()

In [1951]:
# Figure5
for c,col in enumerate(figure5_df.columns):
    figure5_sheet.write(0,c+1,col,heading_format)
    for i,ind in enumerate(figure5_df.index):
        if c==0:
            figure5_sheet.write(i+1,0,figure5_df.index[i],heading_format)
        cell=excel_cols[c+1]+str(i+2)
        figure5_sheet.write_dynamic_array_formula(cell,formula=figure5_df.loc[ind,col], cell_format=percentage_format)
figure5_sheet.set_tab_color('orange')
figure5_sheet.hide()

In [1952]:
# Figure6
for c,col in enumerate(figure6_df.columns):
    figure6_sheet.write(0,c+1,col,heading_format)
    for i,ind in enumerate(figure6_df.index):
        if c==0:
            figure6_sheet.write(i+1,0,figure6_df.index[i],heading_format)
        cell=excel_cols[c+1]+str(i+2)
        figure6_sheet.write_dynamic_array_formula(cell,formula=figure6_df.loc[ind,col], cell_format=percentage_format)
figure6_sheet.set_tab_color('orange')
figure6_sheet.hide()

In [1953]:
# Figure7
for c,col in enumerate(figure7_df.columns):
    figure7_sheet.write(0,c+1,col,heading_format)
    for i,ind in enumerate(figure7_df.index):
        if c==0:
            figure7_sheet.write(i+1,0,figure7_df.index[i],heading_format)
        cell=excel_cols[c+1]+str(i+2)
        figure7_sheet.write_dynamic_array_formula(cell,formula=figure7_df.loc[ind,col], cell_format=scientific_format)
figure7_sheet.set_tab_color('orange')
figure7_sheet.hide()

#### 3.4.19 Dashboard

In [1954]:
waste=general_info['Type of waste']
geo=general_info['Geographical scope']
time=general_info['Temporal scope']
title=f'Treatment of {waste}, {geo}, {time}'

**Text**

In [1955]:
Dashboard_sheet.set_column(0, 200, 100, background_format)
Dashboard_sheet.set_column(0, 0, 50, background_format)
Dashboard_sheet.set_column(1, 1, 60, background_format)


Dashboard_sheet.write('A1',title,title_format)
Dashboard_sheet.write('A9','Environmental Waste Utilization', title_format)
Dashboard_sheet.write('C9','Contribution Analysis', title_format)
Dashboard_sheet.write('P9','Scenario Analysis', title_format)

Dashboard_sheet.write('A3','Pathway/WMS',text_format2)
Dashboard_sheet.write('B3',act_names[0],text_format)
Dashboard_sheet.data_validation('B3', {'validate': 'list',
                                 'source': act_names})

Dashboard_sheet.write('A4','Database',text_format2)
Dashboard_sheet.write('B4',[db.name for db in dbs][0],text_format)
Dashboard_sheet.data_validation('B4', {'validate': 'list',
                                 'source': [db.name for db in dbs]})

Dashboard_sheet.write('A5','Scenario',text_format2)
Dashboard_sheet.write('B5',scenarios[0],text_format)
Dashboard_sheet.data_validation('B5', {'validate': 'list',
                                 'source': scenarios})

Dashboard_sheet.write('A6','Weighting Method',text_format2)
Dashboard_sheet.write('B6',Weighting.columns[0],text_format)
Dashboard_sheet.data_validation('B6', {'validate': 'list',
                                 'source': '=Weighting!B1:'+str(excel_cols[len(Weighting.columns)])+'1'})

Dashboard_sheet.set_zoom(30)
Dashboard_sheet.hide_gridlines(option=2) #hide gridlines on screen and printed

**Figure1**

In [1956]:
chart = workbook.add_chart({'type': 'radar', 'subtype':'filled'})

# Get the number of rows and column index
max_col = len(LCIA_method_names)
col_x = figure1_df.columns.get_loc(LCIA_method_names[0]) + 1
col_y = figure1_df.columns.get_loc(LCIA_method_names[-1]) + 1

max_row = len(act_names)

for i,ind in enumerate(figure5_df.index[:max_row]):
    row_x = figure1_df.index.get_loc(ind) + 1
    chart.add_series({
        'name':       ['EWU_Dashboard', 2, 1, 2, 1],
        'categories': [figure1_sheet_name, 0, col_x, 0, max_col], #[sheetname, first_row, first_col, last_row, last_col]
        'values':     [figure1_sheet_name, row_x, col_x, row_x, col_y],
        'line':       {'color': color_dict[ind]},
        'fill':       {'color': color_dict[ind], 'transparency': 50},
        #'marker':     {'type': 'circle', 'size': 4},
        #'trendline': {'type': 'linear'},
    })
    # Set name on axis
#     chart.set_x_axis({'name': 'Concentration'})
    chart.set_y_axis({'name': 'Measured','major_unit': 1}) #'min': -1, 'max': 2,
                      #'major_gridlines': {'visible': False}},
    chart.set_size({'width': 700, 'height': 370})
    chart.set_legend({'none': True})

    Dashboard_sheet.insert_chart('A10', chart)
    # Close and save the Excel file

**Figure2**

In [1957]:
# Figure 2: Results - figure2_df Aggregated Scores
# Add charts

# Create a chart object.
act_colors=[color_dict[act] for act in act_names]

n=0
for i,ind in enumerate(figure2_df.index[:1]):
    chart = workbook.add_chart({'type': 'column'})
    # Get the number of rows and column index
    max_col = len(LCIA_method_names)
    col_x = figure2_df.columns.get_loc(act_names[0]) + 1
    col_y = figure2_df.columns.get_loc(act_names[-1]) + 1
    row_x = figure2_df.index.get_loc(ind) + 1
    # Create the bar chart
    chart.add_series({
        'name':       ['EWU_Dashboard', 5, 1, 5, 1],
        'categories': [figure2_sheet_name, 0, col_x, 0, max_col], #[sheetname, first_row, first_col, last_row, last_col]
        'values':     [figure2_sheet_name, row_x, col_x, row_x, col_y],
        'points':     [{'fill': {'color': act_color}} for act_color in act_colors],
        #'fill':       {'color': act_colors},
        #'marker':     {'type': 'circle', 'size': 4},
        #'trendline': {'type': 'linear'},
    })
    # Set name on axis
#     chart.set_x_axis({'name': 'Concentration'})
    chart.set_y_axis({'name': 'Indicator Score',})#'min': -1, 'max': 2,'major_unit': 1})
                      #'major_gridlines': {'visible': False}},
    chart.set_size({'width': 700, 'height': 370})
    chart.set_legend({'none': True})

    Dashboard_sheet.insert_chart('A30', chart)
    
    # Close and save the Excel file

**Figure3**

In [1958]:
# Results - Contribution Analysis (Pathways)
# Add charts

# Create a chart object.
chart = workbook.add_chart({'type': 'column', 'subtype': 'stacked'})
max_col = len(LCIA_method_names)
col_x = figure3_df.columns.get_loc(LCIA_method_names[0]) + 1
col_y = figure3_df.columns.get_loc(LCIA_method_names[-1]) + 1
first_row=4
max_row = len(figure3_df.index)+4

for i,ind in enumerate(figure3_df.index[1:]):
    # Get the number of rows and column index

    # Create the scatter plot, use a trendline to fit it
    chart.add_series({
        'name':       ind,
        'categories': [figure3_sheet_name, first_row, col_x, first_row, col_y], #[sheetname, first_row, first_col, last_row, last_col]
        'values':     [figure3_sheet_name, i+first_row+2, col_x, i+first_row+2, col_y],
        'fill':       {'color': color_dict[ind]},
    })
    # Set name on axis
chart.set_x_axis({'label_position':'low'})
chart.set_y_axis({'name': 'Environmental Impact in Person Equivalents'})
                  #'major_gridlines': {'visible': False}},
chart.set_size({'width': 7500, 'height': 370})
#chart.set_legend({'none': True})
chart.set_title({'name':'Contribution Analysis - Treatment Paths'})

line_chart = workbook.add_chart({'type': 'line'})
line_chart.add_series({     
        'name':       [figure3_sheet_name, first_row+1, 0],
        'categories': [figure3_sheet_name, first_row, col_x, first_row, col_y], #[sheetname, first_row, first_col, last_row, last_col]
        'values':     [figure3_sheet_name, first_row+1, col_x, first_row+1, col_y],
        'line':       {'color': 'black','transparency':100},
        'marker':     {'border': {'color':'black'}, 'fill': {'color':'black'},'size':5,'type': 'diamond'}
        })

chart.combine(line_chart)

Dashboard_sheet.insert_chart('C10', chart)

0

**Figure4**

In [1959]:
# Results - Contribution Analysis (Pathways)
# Add charts

# Create a chart object.
chart = workbook.add_chart({'type': 'column', 'subtype': 'stacked'})
max_col = len(LCIA_method_names)
col_x = figure4_df.columns.get_loc(LCIA_method_names[0]) + 1
col_y = figure4_df.columns.get_loc(LCIA_method_names[-1]) + 1
first_row=4
max_row = len(figure4_df.index)+4

for i,ind in enumerate(figure4_df.index):
    # Get the number of rows and column index

    # Create the scatter plot, use a trendline to fit it
    chart.add_series({
        'name':       ind,
        'categories': [figure4_sheet_name, first_row, col_x, first_row, col_y], #[sheetname, first_row, first_col, last_row, last_col]
        'values':     [figure4_sheet_name, i+first_row+1, col_x, i+first_row+1, col_y],
        'fill':       {'color': color_dict[ind]},
    })
    # Set name on axis
chart.set_x_axis({'label_position':'low'})
chart.set_y_axis({'name': 'Environmental Impact in Person Equivalents'})
                  #'major_gridlines': {'visible': False}},
chart.set_size({'width': 7500, 'height': 370})
#chart.set_legend({'none': True})
chart.set_title({'name':'Contribution Analysis - Activities'})

line_chart = workbook.add_chart({'type': 'line'})
line_chart.add_series({     
        'name':       [figure3_sheet_name, first_row+1, 0],
        'categories': [figure3_sheet_name, first_row, col_x, first_row, col_y], #[sheetname, first_row, first_col, last_row, last_col]
        'values':     [figure3_sheet_name, first_row+1, col_x, first_row+1, col_y],
        'line':       {'color': 'black','transparency':100},
        'marker':     {'border': {'color':'black'}, 'fill': {'color':'black'},'size':5,'type': 'diamond'}
        })

chart.combine(line_chart)

Dashboard_sheet.insert_chart('C30', chart)

0

**Figure5**

In [1960]:
# Figure 5: Results - figure5_df Aggregated Scores
# Add charts

# Create a chart object.
act_colors=[color_dict[act] for act in act_names]

n=0
# Create a chart object.
chart = workbook.add_chart({'type': 'column', 'subtype': 'stacked'})
col_x = 1
col_y = len(figure5_df.columns)
first_row=0
max_row = len(act_names)

for i,ind in enumerate(figure5_df.index[:max_row]):
    # Get the number of rows and column index

    # Create the scatter plot, use a trendline to fit it
    chart.add_series({
        'name':       ind,
        'categories': [figure5_sheet_name, first_row, col_x, first_row, col_y], #[sheetname, first_row, first_col, last_row, last_col]
        'values':     [figure5_sheet_name, i+first_row+1, col_x, i+first_row+1, col_y],
        'fill':       {'color': color_dict[ind]},
    })
    # Set name on axis
#     chart.set_x_axis({'name': 'Concentration'})
chart.set_y_axis({'name': 'Indicator Score',})#'min': -1, 'max': 2,'major_unit': 1})
                  #'major_gridlines': {'visible': False}},
chart.set_size({'width': 7500, 'height': 370})
chart.set_legend({'none': True})
chart.set_title({'name': 'EWU_Dashboard!B3'})

Dashboard_sheet.insert_chart('P10', chart)
    
    # Close and save the Excel file

0

**Figure6**

In [1961]:
# Figure 6: Results - figure6_df Aggregated Scores
# Add charts

# Create a chart object.
act_colors=[color_dict[act] for act in act_names]

n=0
# Create a chart object.
chart = workbook.add_chart({'type': 'column', 'subtype': 'stacked'})
col_x = 1
col_y = len(figure6_df.columns)
first_row=0
max_row = len(act_names)

for i,ind in enumerate(figure6_df.index[:max_row]):
    # Get the number of rows and column index

    # Create the scatter plot, use a trendline to fit it
    chart.add_series({
        'name':       ind,
        'categories': [figure6_sheet_name, first_row, col_x, first_row, col_y], #[sheetname, first_row, first_col, last_row, last_col]
        'values':     [figure6_sheet_name, i+first_row+1, col_x, i+first_row+1, col_y],
        'fill':       {'color': color_dict[ind]},
    })
    # Set name on axis
#     chart.set_x_axis({'name': 'Concentration'})
chart.set_y_axis({'name': 'Indicator Score',})#'min': -1, 'max': 2,'major_unit': 1})
                  #'major_gridlines': {'visible': False}},
chart.set_size({'width': 7500, 'height': 370})
chart.set_legend({'none': True})
chart.set_title({'name': 'EWU_Dashboard!B3'})

Dashboard_sheet.insert_chart('P30', chart)
    
    # Close and save the Excel file

0

**Figure7**

In [1962]:
# Figure 7: Results - figure7_df Aggregated Scores
# Add charts

# Create a chart object.
act_colors=[color_dict[act] for act in act_names]

n=0
# Create a chart object.
chart = workbook.add_chart({'type': 'column'})
col_x = 1
col_y = len(figure7_df.columns)
first_row=0
max_row = len(act_names)

for i,ind in enumerate(figure7_df.index[:max_row]):
    # Get the number of rows and column index

    # Create the scatter plot, use a trendline to fit it
    chart.add_series({
        'name':       ind,
        'categories': [figure7_sheet_name, first_row, col_x, first_row, col_y], #[sheetname, first_row, first_col, last_row, last_col]
        'values':     [figure7_sheet_name, i+first_row+1, col_x, i+first_row+1, col_y],
        'fill':       {'color': color_dict[ind]},
    })
chart.set_x_axis({'label_position':'low'})
chart.set_y_axis({'name': 'Environmental Impact in Person Equivalents',})#'min': -1, 'max': 2,'major_unit': 1})
                  #'major_gridlines': {'visible': False}},
chart.set_size({'width': 700, 'height': 370})
chart.set_legend({'none': True})
chart.set_title({'name': 'EWU-Components'})

Dashboard_sheet.insert_chart('A50', chart)
    
    # Close and save the Excel file

0

### 3.5 Save & Close 

In [1963]:
writer.save()
writer.close()

### 3.6 Insert User Guide and Glossar

In [1964]:
from win32com.client import Dispatch

path1 = os.path.abspath('ExcelTool_GeneratorInput_Template.xlsx')
path2 = os.path.abspath(EWU_dashboard)

xl = Dispatch("Excel.Application")
xl.Visible = True

wb1 = xl.Workbooks.Open(Filename=path1)
wb2 = xl.Workbooks.Open(Filename=path2)

ws1 = wb1.Worksheets("Glossary")
ws2 = wb1.Worksheets("UserGuide_EWU-Dashboard")
ws1.Copy(Before=wb2.Worksheets(1))
ws2.Copy(Before=wb2.Worksheets(1))

wb2.Close(SaveChanges=True)
#xl.Quit()